In [1]:
# This code is a recreation of the original tube making code in Python. 
# The input is a magno video and the output is a list of tubes and an output video with tubes overlaid.
# Here, we break the output up into "stages," which build upon each other 


# Necessary inputs
import cv2
import numpy as np
import argparse
import csv
import os
import math
import random
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.cluster import DBSCAN
from copy import deepcopy
from operator import itemgetter
from collections import Counter
from itertools import *


In [9]:
# Useful Functions
def make_ellipse(image, row):
    center = (int(round(row[0])),int(round(row[1])))
    axes = (int(round(row[2])*2 ),int(round(row[3])*2) )
    angle = int(round(row[4]))
    color = (row[8],row[9],row[10])
    thickness = 4  #Do -1 if you want it filled in

    return cv2.ellipse(image, center, axes, angle, 0, 360, color, thickness) 

def make_ellipse2(image, ellipse):
    thickness = 4  #Do -1 if you want it filled in
    
    return cv2.ellipse(image, ellipse[1], ellipse[3], thickness) 

def make_ellipse3(image, ellipse,tubenum):
    thickness = 4  #Do -1 if you want it filled in
    center = (int(ellipse[0][1][0][0]), int(ellipse[0][1][0][1]) )
    
    cv2.putText(image,str(tubenum), center,cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), thickness)
    return cv2.ellipse(image, ellipse[0][1], ellipse[0][3], thickness) 

def draw_event(image, event):
    thickness = 4
    center = (int(event[2]), int(event[4]) )
    
    return cv2.putText(image,"X", center,cv2.FONT_HERSHEY_PLAIN, 4, (0,0,255), thickness)

def writeTubelist(tubelistLoc, tubes):
    with open(tubelistLoc, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["f","x","y","b","a","phi","Vx","Vy","R","G","B","T"])

        for tube in tubes:
            for ell in tube:
                writer.writerow([ell[0],ell[1][0][0],ell[1][0][1],ell[1][1][0],ell[1][1][1],ell[1][2],ell[2][0],ell[2][1],ell[3][0],ell[3][1],ell[3][2],tubes.index(tube) ])

def readTubelist(tubelistLoc):
        with open(tubelistLoc) as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            next(readCSV) #burn the first line
            tubes2 = []
            
            for row in readCSV:
                row = [float(i) for i in row] #text-->float for each thingy
                tubes2.append(row) #Read into intermediate var
                
        numtubes = tubes2[-1][-1]+1
        tubes = []
        for i in range(0,int(numtubes)):
            thistube = [ [e[0],((e[1],e[2]),(e[3],e[4]),e[5]),[e[6],e[7]],[e[8],e[9],e[10]] ] for e in tubes2 if e[-1]==i]
            tubes.append(thistube)
        
        return tubes
                

def getBoxFromEllipse(e):
    # Bounding box is normally (xmin, ymin, width, height) 
    bbox = [int(e[1][0][0]-e[1][1][0]/2), int(e[1][0][1] - e[1][1][1]/2), int(e[1][1][0]), int(e[1][1][1]) ]
    if bbox[0]<0:
        bbox[0]=1
    if bbox[1]<0:
        bbox[1]=1
    return tuple(bbox)

def getEllipseFromBox(b, rot=0):
    # Ellipse is ((centerx, centery), (smallax, bigax), degrotation)
    # maybe pass in rotation
    center = (b[0]+b[2]/2 , b[1]+b[3]/2)
    size = (b[2], b[3])
    ell = (center, size, rot)
    
    return ell

def writevid(vid, outvidpath):
     #Look at imageshape output and take size from there (reversed)
    H = vid[1].shape[0]
    W = vid[1].shape[1]
    numframes = len(vid)
    vidwriter = cv2.VideoWriter(outvidpath, cv2.VideoWriter_fourcc(*'MP4V'),10, (W, H))

    for i in range(1, numframes):
        if i%50==0:
            print("FRAME" , i, "/", numframes)        
        vidwriter.write(vid[i])
               
    vidwriter.release()
    cv2.destroyAllWindows()
    
def readvid(vidinfile):
    #Read in video at vidinfile
    vidcap = cv2.VideoCapture(vidinfile) 
    thevid = []
    F=0
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            thevid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()
    print(type(thevid))
    return thevid

def findEdges(vid):
    frame = vid[20]
    A = frame[:,0,:]
    i=1
    while np.all(A==0):
        A = frame[:,i,:]
        i+=1
    left = i
    B = frame[:,-1,:]
    i=1
    while np.all(B==0):
        B = frame[:,-i,:]
        i+=1
    right= W-i
    C = frame[0,:,:]
    i=1
    while np.all(C==0):
        C = frame[i,:,:]
        i+=1
    top = i
    D = frame[-1,:,:]
    i=1
    while np.all(D==0):
        D = frame[-i,:,:]
        i+=1
    bottom = H-i
    #print([left, right, top, bottom])
    if [left, right,top, bottom] == [1,-1,1,-1]:
        return [1,W,1,H]
    return [left, right, top, bottom]
    
    
def recalcVelocity(tubes, Vgap):
    #Vgap = 10  #calculating velocities across +/- Vgap 
    for t in tubes:
        for i in range(0,len(t)): #for each ellipse
            close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
            starter = close[0]
            ender = close[-1]
            fgap = ender[0]-starter[0]
            Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
            Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
            t[i][2] = [Vx, Vy]

def putTracksOnVid(tracks, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Put the red rectangles on the vid for the tracks
    outvid = deepcopy(inputvid)
    for f in range(1,len(outvid)):
        here = []
        for T in tracks:
            here.extend([t for t in T if t[0]==f])
        if here: #if any track has a box on this frame
            for h in here:
                bbox = h[1]
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(outvid[f], p1, p2, (0,0,255), 2, 1) #red
            
    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()
    
def putTracksOnVid2(tracks, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Put the red rectangles on the vid for the tracks
    outvid = deepcopy(inputvid)
    for f in range(1,len(outvid)):
        here = []
        here.extend([t for t in tracks if t[0]==f])
        if here: #if any track has a box on this frame
            for h in here:
                bbox = h[1]
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(outvid[f], p1, p2, (0,0,255), 2, 1) #red
            
    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()    
    
def putMarksOnVid(marks, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Check score for mark and put proper thing on the vid 
    outvid = deepcopy(inputvid)
    for f in range(1,len(outvid)):
        here = []
        for T in marks:
            here.extend([t for t in T if t[0]==f])
        if here: #if any track has a box on this frame
            for h in here:
                bbox = h[1]
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                if h[2]==1:
                    #cv2.rectangle(outvid[f], p1, p2, (0,255,255), 2, 6) #yellow
                    continue
                elif h[2]==2:
                    cv2.rectangle(outvid[f], p1, p2, (0,0,255), 2, 10) #red
                else:
                    cv2.rectangle(outvid[f], p1, p2, (0,255,0), 2, 10) #green
            
    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()
    
    
def putTubesOnVid(tubevar, vidoutfile, withnums = False):
    
    if not withnums:
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                puthere.extend(P)

            for ellipse in puthere:
                make_ellipse2(outvid[f],ellipse)

        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    else:        
        #WITH NUMS    
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                T = [tubevar.index(path) for e in path if e[0]==f]
                puthere.append([P,T])

            for [ellipse,Tnum] in puthere:
                if ellipse:
                    make_ellipse3(outvid[f],ellipse,Tnum)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()


def putSomeTubesOnVid(tubevar, vidoutfile, which =[], withnums = True):
    
    if withnums:
        #WITH NUMS    
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        selectedtubes = [tubevar[w] for w in which]

        for f in range(1,F):
            puthere = []

            for path in selectedtubes:
                P = [e for e in path if e[0]==f]
                T = [tubevar.index(path) for e in path if e[0]==f]
                puthere.append([P,T])

            for [ellipse,Tnum] in puthere:
                if ellipse:
                    make_ellipse3(outvid[f],ellipse,Tnum)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
        
    else:
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        selectedtubes = [tubevar[w] for w in which]
        for f in range(1,F):
            puthere = []

            for path in selectedtubes:
                P = [e for e in path if e[0]==f]
                puthere.extend(P)

            for ellipse in puthere:
                make_ellipse2(outvid[f],ellipse)

        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    

def putEventsOnVid(eventvar, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Draw the events on the video to get a visual
    outvid = deepcopy(inputvid)
    for f in range(1,F):
        puthere = [e for e in eventvar if (f>=e[0]) and (f<=e[1])]
        for event in puthere:
            draw_event(outvid[f],event)

    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()

%matplotlib notebook
def plotTubes(tubes, which = [],zlim=None,colorMatch=True):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    colors = ['r','g','b','c','m','y','k','xkcd:hot pink','xkcd:sand','xkcd:mint','xkcd:wine red']
    ax.set_xlabel('Time')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')
    if zlim:
        ax.set_zlim(zlim)
    
    if not(which):
        for t in tubes:
            Ts = [e[0] for e in t]
            Xs = [e[1][0][0] for e in t]
            Ys = [e[1][0][1] for e in t]
            RGB = [x/255 for x in t[0][3]]
            RGB.reverse()
            #Cs = t[0][3]
            #ax.scatter(Ts,Xs,Ys,c=random.choice(colors), marker='.')
            ax.scatter(Ts,Xs,Ys,c=[RGB], marker='.')

    else:
        for w in set(which):
            tubeToPlot = tubes[w]
            Ts = [e[0] for e in tubeToPlot]
            Xs = [e[1][0][0] for e in tubeToPlot]
            Ys = [e[1][0][1] for e in tubeToPlot]
            RGB = [x/255 for x in tubeToPlot[0][3]]
            RGB.reverse()
            print(RGB)
            #Cs = [e[3] for e in tubeToPlot]
            #ax.scatter(Ts,Xs,Ys,c=colors[which.index(w)], marker='.')
            ax.scatter(Ts,Xs,Ys,c=[RGB], marker='.')
            
    plt.show()
    return plt
    
    
# Tube merging functions to prepare us for the next stage(s)

def getMerges(L, lookahead):
    mergeList = []
    for i in range(0,len(L)):
        
        if len(L[i]) < 10:
            print("Too short")
            continue
            
        CVx = [L[i][ind][2][0] for ind in range(-5,0)]
        CVy = [L[i][ind][2][1] for ind in range(-5,0)]
        forVx = sum(CVx)/len(CVx)
        forVy = sum(CVy)/len(CVy)  #ending velocity of tube i
        BVx = [L[i][ind][2][0] for ind in range(0,5)]
        BVy = [L[i][ind][2][1] for ind in range(0,5)]
        backVx = sum(BVx)/len(BVx)
        backVy = sum(BVy)/len(BVy)  #starting velocity of tube i

        
        
        forX = [L[i][ind][1][0][0] for ind in range(-10,0)]
        forY = [L[i][ind][1][0][1] for ind in range(-10,0)]
        backX = [L[i][ind][1][0][0] for ind in range(0,10)]
        backY = [L[i][ind][1][0][1] for ind in range(0,10)]
        
        forVx = (forX[-1] - forX[0]) /10
        forVy = (forY[-1] - forY[0]) /10
        backVx = (backX[-1] - backX[0]) /10
        backVy = (backY[-1] - backY[0]) /10

        predx = (sum(forX)/len(forX)) + (forVx*lookahead)
        predy = (sum(forY)/len(forY)) + (forVy*lookahead)
        predf = L[i][-1][0] + lookahead
        backpredx = (sum(backX)/len(backX)) - (backVx*lookahead)
        backpredy = (sum(backY)/len(backY)) - (backVy*lookahead)
        backpredf = L[i][0][0] - lookahead

        for j in range(0,len(L)):
            if i==j:
                continue

            closeones = [e for e in L[j] if abs(e[0] - predf)<5]
            Bcloseones= [e for e in L[j] if abs(e[0]-backpredf)<5]
            matchFlag = False
            BmatchFlag= False

            for e in closeones:
                r = math.sqrt((e[1][0][0] - predx)**2 + (e[1][0][1] - predy)**2 )
                if (r<100) and (abs(e[2][0] - forVx)<2) and (abs(e[2][1] - forVy)<2):
                    matchFlag = True

            for e in Bcloseones:
                r = math.sqrt((e[1][0][0] - backpredx)**2 + (e[1][0][1] - backpredy)**2 )
                if (r<50) and (abs(e[2][0] - backVx)<2) and (abs(e[2][1] - backVy)<2):
                    BmatchFlag = True

            if (matchFlag):
                mergeList.append([i,j,0])  #j goes in back of i
            if (BmatchFlag):
                mergeList.append([i,j,1])  #j goes in front of i

    nahList = []
    for m in mergeList:
        a = m[0]
        b = m[1]
        
        allVxa = [L[a][ind][2][0] for ind in range(0,len(L[a])) ] 
        Vxa = sum(allVxa)/len(allVxa)
        allVya = [L[a][ind][2][1] for ind in range(0,len(L[a])) ] 
        Vya = sum(allVya)/len(allVya)  

        allVxb = [L[b][ind][2][0] for ind in range(0,len(L[b])) ] 
        Vxb = sum(allVxb)/len(allVxb)
        allVyb = [L[b][ind][2][1] for ind in range(0,len(L[b])) ] 
        Vyb = sum(allVyb)/len(allVyb)

        if (Vxa/Vxb>0) and (Vya/Vyb>0 or (abs(Vya)<1.2 and abs(Vyb)<1.2) ) :
            A = [L[a][ind][0] for ind in range(0,len(L[a]))]
            B = [L[b][ind][0] for ind in range(0,len(L[b]))]
            overlap = [val for val in A if val in B]
            if len(overlap) > min(len(A),len(B))/4:
                nahList.append(m)
        else:
            nahList.append(m)

    mergeList = [m for m in mergeList if m not in nahList]
    
    return mergeList

def doMerge(L, mergeList):
    
    firtubes = [m[0] for m in mergeList]
    sectubes = [m[1] for m in mergeList]
    two2oneA = [t for t in range(0,len(L)) if firtubes.count(t)>1]
    two2oneB = [t for t in range(0,len(L)) if sectubes.count(t)>1]
    #delList = [m for m in mergeList if (m[0] in two2oneA) or (m[1] in two2oneB)]
    #mergeList = [m for m in mergeList if m not in delList]
    #print("New mergeList:", mergeList)
    
    colorList = [t[0][3] for t in L]
    for m in mergeList:
        tlist = []
        for i in range(0,len(L)):
            for e in L[i]:
                if e[3]==colorList[m[0]] or e[3]==colorList[m[1]]:
                    tlist.append(i)

        flattubes = [ell for tube in L for ell in tube]
        cList = [colorList[a] for a in set(tlist)]
        connected = [e for e in flattubes if e[3] in cList]
        for i in set(tlist):
            L[i] = connected
    
    #Remove the extras (duplicates)
    delList = []
    for i in range(0,len(L)):
        if not(L.index(L[i]) == i):
            delList.append(i)
            
    #print(delList)
    Otubes = [L[i] for i in range(0,len(L)) if i not in delList ]
    
    for t in Otubes:
        for e in t:
            e[3] = t[0][3]
    

    return Otubes



def compareImg(img1,img2,useorb=True,useflann=0):
    
    if useorb:
        #-- Step 1A: Detect the keypoints using ORB Detector, compute the descriptors
        detector = cv2.ORB_create()
        keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
        keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
    else:
        #-- Step 1B: Detect the keypoints using SURF Detector, compute the descriptors
        minHessian = 500 #big number = more 'choosy'
        detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
        keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
        keypoints2, descriptors2 = detector.detectAndCompute(img2, None)

        
    #Check if there are keypoints
    #print("Num KPS:", len(keypoints1), len(keypoints2))
    if len(keypoints1)==0 or len(keypoints2)==0:
        #If not, do a direct image comparison
        print("Don't see any KPs here buddy.")
        newH = round((img1.shape[0]+img2.shape[0])/2)
        newW = round((img1.shape[1]+img2.shape[1])/2)
        newimg1 = cv2.resize(img1, dsize=(newW, newH), interpolation=cv2.INTER_CUBIC)
        newimg2 = cv2.resize(img2, dsize=(newW, newH), interpolation=cv2.INTER_CUBIC)
        #print("NORM:", np.linalg.norm( np.subtract(newimg1,newimg2)) )
        res = np.linalg.norm( np.subtract(newimg1,newimg2))
        
    else:
        if useflann==0:
            #-- Step 2A: Matching descriptor vectors with a Brute Forcer
            if useorb:
                bf = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
            else:
                bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=True)
            bfmatches = bf.match(descriptors1,descriptors2)
            bfmatches = sorted(bfmatches, key = lambda x:x.distance)
            result = cv2.drawMatches(img1,keypoints1,img2,keypoints2,bfmatches,None)  
            print("Num matches:", len(bfmatches))
            print("Percent = ", len(bfmatches)/len(keypoints1)*100)
            res = len(bfmatches)/len(keypoints1)
            
        elif useflann==1:    
            #-- Step 2B: Matching descriptor vectors with a FLANN based matcher
            # Since SURF is a floating-point descriptor NORM_L2 is used
            matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
            knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)

            #-- Filter matches using the Lowe's ratio test
            good_matches = []
            for m,n in knn_matches:
                if m.distance < 0.7 * n.distance:
                    good_matches.append(m)

            img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
            result = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            #print("Num matches:", len(good_matches))
            #print("Percent = ", len(good_matches)/len(keypoints1)*100)
            res = len(good_matches)/len(keypoints1)
            
        #-- Show detected matches
        #cv2.imshow('Good Matches', result)
        #cv2.waitKey(0)
        
        

    cv2.destroyAllWindows()
    return res
    
def getImgFromEllipse(E):
    B = getBoxFromEllipse(E)
    imgA = V[int(E[0])][B[1]:B[1]+B[3],B[0]:B[0]+B[2]]
    
    #cv2.imshow('Img from ellipse', imgA)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return imgA

def compareImgFromEllipse(E1,E2=None,useorb=True,useflann=0):
    img1 = getImgFromEllipse(E1)
    if not(E2):
        img2 = V[int(E1[0])]
        res = compareImg(img1,img2,useorb,useflann)
        return res
    else:
        img2 = getImgFromEllipse(E2)
        res = compareImg(img1,img2,useorb,useflann)
        return res
            
def getMotionScores(T1, T2):
    #for now... returns
    # [Xscore,Yscore,Locscore,Xcos,Ycos,Vxfir,Vxsec,Vyfir,Vysec]
    if (T1[round(len(T1)/2)][0] < T2[round(len(T2)/2)][0]):
        isT2After = True
    else:
        isT2After = False
        
    #Spatial prediction (going in A frames deep either side)
    A = 10 #KEEP THIS AT 10 PLEASE. YOU'LL BREAK STUFF AND WANT TO CRY IF YOU DON'T
    predf = T2[A][0]
    R = (T1[-A][1][1][0]+T1[-A][1][1][1])/2
    if predf in [e[0] for e in T1]:
        #Unlikely but if so, use this as the prediction
        fgap=1
        predX = [e for e in T1 if e[0]==predf][0][1][0][0]
        predY = [e for e in T1 if e[0]==predf][0][1][0][1]
    else:
        fgap = abs(T2[A][0]-T1[-A][0])
        predX = T1[-A][2][0]*fgap + T1[-A][1][0][0]
        predY = T1[-A][2][1]*fgap + T1[-A][1][0][1]
    diffX = abs(T2[A][1][0][0] - predX)
    diffY = abs(T2[A][1][0][1] - predY)
    diff = math.sqrt(diffX**2 +  diffY**2)
    Xtrav = abs(T1[-1][1][0][0] - T1[0][1][0][0])
    Ytrav = abs(T1[-1][1][0][1] - T1[0][1][0][1])
    ExpX = T1[-A][2][0] * fgap
    ExpY = T1[-A][2][1] * fgap
    #Xscore = math.exp(-(diffX/math.sqrt(ExpX**2+ExpY**2)))
    #Yscore = math.exp(-(diffY/math.sqrt(ExpX**2+ExpY**2)))
    #Locscore = math.exp(-(diff/math.sqrt(ExpX**2+ExpY**2)))
    Xscore = math.exp(-(diffX/math.sqrt(Xtrav**2+Ytrav**2)))
    Yscore = math.exp(-(diffY/math.sqrt(Xtrav**2+Ytrav**2)))
    #Locscore = diff
    #Locscore = math.exp(-diff/(T1[-A][1][1][1]))
    Locscore = math.exp(-(diff/math.sqrt(Xtrav**2+Ytrav**2)))
    #print("X score is:", diffX, Xscore)
    #print("Y score is:", diffY, Yscore)
    #print("Loc score is:",diff,Locscore)
    
    
    #Cosine directional alignment
    thisT = np.array([e[0] for e in T1])
    thisX = np.array([e[1][0][0] for e in T1])
    thisY = np.array([e[1][0][1] for e in T1])
    nextT = np.array([e[0] for e in T2])
    nextX = np.array([e[1][0][0] for e in T2])
    nextY = np.array([e[1][0][1] for e in T2])
    thisXofT = np.polyfit(thisT,thisX,1)
    thisYofT = np.polyfit(thisT,thisY,1)
    nextXofT = np.polyfit(nextT,nextX,1)
    nextYofT = np.polyfit(nextT,nextY,1)
    Xangle = np.arctan(abs((nextXofT[0]-thisXofT[0]) / (1 + nextXofT[0]*thisXofT[0]))) 
    Xcos = np.cos(Xangle)
    Yangle = np.arctan(abs((nextYofT[0]-thisYofT[0]) / (1 + nextYofT[0]*thisYofT[0]))) 
    Ycos = np.cos(Yangle)
    #print("X cosine score", Xcos)
    #print("Y cosine score", Ycos)
    
    #Velocity comparison
    Vx1 = [e[2][0] for e in T1]
    Vy1 = [e[2][1] for e in T1]
    Vx2 = [e[2][0] for e in T2]
    Vy2 = [e[2][1] for e in T2]
   #Vxfir = ((sum(Vx1)/len(Vx1)) + (sum(Vx1[-A:])/A))/2
   #Vxsec = ((sum(Vx2)/len(Vx2)) + (sum(Vx2[-A:])/A))/2
   #Vyfir = ((sum(Vy1)/len(Vy1)) + (sum(Vy1[-A:])/A))/2
   #Vysec = ((sum(Vy2)/len(Vy2)) + (sum(Vy2[-A:])/A))/2
    
    Vxfir =sum(Vx1[-A:])/A
    Vxsec =sum(Vx2[:A])/A
    Vyfir =sum(Vy1[-A:])/A
    Vysec =sum(Vy2[:A])/A
    
    
    #print("Vx matchup", Vxfir, Vxsec)
    #print("Vy matchup", Vyfir, Vysec)
    
    #print("Perc diff Vx:", abs((Vxsec - Vxfir)/Vxfir) )
    #print("Perc diff Vy:", abs((Vysec - Vyfir)/Vyfir) )

    scores = [Xscore, Yscore, Locscore, Xcos, Ycos, Vxfir, Vxsec, Vyfir,Vysec]
    return scores
    

 

In [14]:
# Stage 0: This stage shows the ellipses before any temporal associations are made. 
# We already filter out the smallest (<20 pix) ellipses before including them in our eList

vidbase = 'ucr3'
inmagnovid = vidbase + '_filtmagno.mp4'

V = readvid(vidbase+".mp4")
vidcap = cv2.VideoCapture(inmagnovid) 
kernel = np.ones((10,10) , np.uint8)
magnovid = []
H=0
W=0
F=0

#Thresholds (at 50) the input magno video into a binary video 'magnovid'
while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        (dontcare, outframe) = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY)
        cv2.dilate(outframe,kernel, iterations =1)
        cv2.erode(outframe,kernel, iterations =1)
        H = frame.shape[0]
        W = frame.shape[1]
        F=F+1
        magnovid.append(outframe)

vidcap.release()
cv2.destroyAllWindows()


##Write the input video into a variable
invideo = vidbase + '.mp4'
vidcap = cv2.VideoCapture(invideo) 
invid = []
F2 = 0

while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        F2=F2+1
        invid.append(frame)
        
vidcap.release()
cv2.destroyAllWindows()
F = min(F,F2)

#Writes the thresholded magno vid to a video file
vidwriter = cv2.VideoWriter(vidbase +'_magnoOut.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%100==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnovid[i])
vidwriter.release()
cv2.destroyAllWindows()


START = time.time()
#--------------------------------------------START TIMING HERE-----------------------------------

#Each ellipse is put in the list as a tuple... (framenum, ellipse)
#Ellipse: (x,y) , (MAJORAXIS,minoraxis), angle
eList = []
for f in range(1,F): #for each frame
    M = cv2.cvtColor(magnovid[f], cv2.COLOR_BGR2GRAY);
    contours, hierarchy = cv2.findContours(M, 1, 2)
    
    for c in contours:
        if len(c) < 5:
            continue
        area = cv2.contourArea(c)
        if area < 100:
            continue
    
        ellipse = cv2.fitEllipse(c)
        RR = random.randint(0,255)
        GG = random.randint(0,255)
        BB = random.randint(0,255)
        eList.append( [f, ellipse, [0,0], [RR,GG,BB] ] )
        
print(len(eList))
MAXS = np.asarray([max(e[1][1][0],e[1][1][1]) for e in eList if e[0] > 10])
MINS = np.asarray([min(e[1][1][0],e[1][1][1]) for e in eList if e[0] > 10])
AVGS = np.asarray([(e[1][1][0]+e[1][1][1])/2  for e in eList if e[0] > 10])

#Filtering the smallest ellipses (minsize for either dimension = 20pixels)



minsize =  .30 * np.mean(AVGS)
#minsize = 10
eList = [e for e in eList if (e[1][1][0] > minsize) and (e[1][1][1]>minsize)]


#
##Draw the ellipses on the video just to check...
#magnotubes = deepcopy(V)
#for f in range(1,F):
#    puthere = [e for e in eList if e[0]==f]
#    
#    for ellipse in puthere:
#        make_ellipse2(magnotubes[f],ellipse)
#        
#
##Writes the vid with tubes to a video file
#vidwriter = cv2.VideoWriter(vidbase+'_outstage0NEWMINSIZE.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
#for i in range(1, F):
#    if i%100==0:
#        print("FRAME" , i, "/", F)        
#    vidwriter.write(magnotubes[i])
#vidwriter.release()
#cv2.destroyAllWindows()

# Stage 1: We make the first swipe at temporal associations, turning ellipses into paths.

#Doing the first temporal associations. Should start to look like tubes
initialList = []
finalList = []
paths = [[]]
istart = 6
iend = istart+4
fstart = 11
fend = fstart+5
locthresh = 80
sizethresh = 50

while (fend < F):
    initialList = [e for e in eList if e[0]>=istart and e[0]<=iend]
    finalList =  [e for e in eList if e[0]>=fstart and e[0]<=fend]
    
    for ei in initialList:
        for ef in finalList:
            
            matchFlag = False
            r = math.sqrt((ef[1][0][0] - ei[1][0][0])**2 + (ef[1][0][1] - ei[1][0][1])**2 )
            sizediff = ef[1][1][1] - ei[1][1][1] #diff in major axis
            framediff = ef[0]-ei[0]
            
            if not(ei[2]==(0,0)):
                predX = ei[2][0]*framediff + ei[1][0][0]
                predY = ei[2][1]*framediff + ei[1][0][1]
                if (abs(predX-ef[1][0][0]) < 50) and (abs(predY-ef[1][0][1]) < 50):
                    matchFlag = True
            else:
                if (abs(sizediff)<sizethresh) and (r<locthresh):
                    matchFlag = True
            
            if (matchFlag):
                if ei[3]==[255,255,255]:
                    ei[3][0] = random.randint(0,255)
                    ei[3][1] = random.randint(0,255)
                    ei[3][2] = random.randint(0,255)
                ef[3][0] = ei[3][0]
                ef[3][1] = ei[3][1]
                ef[3][2] = ei[3][2]
                
                ef[2][0] = (ef[1][0][0]-ei[1][0][0])/framediff
                ef[2][1] = (ef[1][0][1]-ei[1][0][1])/framediff
                

                #This part searches for ei in existing paths and attaches ef if it finds it
                whichpath = 0
                for i in range(1,len(paths)): 
                    for ell in paths[i]:
                        if ei == ell:
                            whichpath = i
                            
                if not(whichpath==0):
                     paths[whichpath].append(ef)
                else:
                    newpath = []
                    newpath.append(ei)
                    newpath.append(ef)
                    paths.append(newpath)

    #print(fend)
    istart=istart+5
    iend=iend+5
    fstart=fstart+5
    fend=fend+5


    
stage1tubes = paths 
#----------------------------------------------------------------------------------------------------
# Stage 2: If we have 2+ ellipses in the same path on the same frame, chose the larger one

#Fixes the paths so that each color corresponds to a path
newpaths=[[]]

for i in range(1,len(paths)):
    for e in paths[i]:
        RGB = e[3]
        makePathFlag = True
        
        for newpath in newpaths:
            if not newpath or not(newpath[0][3] == RGB):
                continue
            else:
                newpath.append(e)
                makePathFlag = False
                break
        
        if (makePathFlag):
            E = [e]
            newpaths.append(E)

            
#Gets rid of duplicates in each path (took forever smh)
K = deepcopy(newpaths)
newpaths = []

for i in range(0,len(K)):
    bigA = []
    for j in range(0,len(K[i])):
        thise = K[i][j]
        A = [K[i].index(e) for e in K[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [K[i][t] for t in list(set(bigA))]
    newpaths.append(out)

#Make sure newpaths is in frame order
for P in newpaths:
    P.sort(key = lambda x: x[0])
    
pre2tubes = newpaths

#-------------------------------------------------------------------------------
#Handles when there are 2 ellipses from the same tube on a single frame.
L = deepcopy(pre2tubes)

tubesToAdd = []
#toRemove = []

for i in range(0,len(L)):
    tubeSplits = []
    R =np.mean(AVGS)
    AList = []
        
    for f in range(1,F):
        A = [e for e in L[i] if e[0] == f] #list of ellipses on this frame in this tube
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) #biggest is last
            AList.append(A)
            
            
    #Check if AList should be split
    Asave = []
    maxlen=0
    Acount =0
    #if i==30:
     #   print(AList)
    for A in AList:
        maxR = max([e[1][1][0] for e in A])
        distList = [math.sqrt((a[1][0][0]-b[1][0][0])**2 + (a[1][0][1] - b[1][0][1])**2) for a in A for b in A if a!=b]
        if any(d>1.5*maxR for d in distList) :
            Acount=Acount+1
        if len(A) > maxlen:
            maxlen = len(A)
            Asave = A
    if i==14:
        print("Tube14")
        print(Acount)
        print(len(AList))
    if (Acount > 0.6*len(AList)) and (Acount!=0):  #then it should be split
        tubeSplits = [[k] for k in Asave]
        startind = AList.index(Asave)
        
        thisind= startind
        
        #Double while to fill in before and after the temporal location with most splits
        while (thisind < len(AList)-1):
            thisind = thisind+1
            lastind = thisind
            for a in AList[thisind]:
                dists = [math.sqrt((a[1][0][0]-tubeSplits[k][-1][1][0][0])**2 + (a[1][0][1] - tubeSplits[k][-1][1][0][1])**2) for k in range(0,len(tubeSplits))]
                #if min(dists) < 400:
                tubeSplits[dists.index(min(dists))].append(a)
                
        thisind=startind    
        while (thisind > 0):
            thisind = thisind-1
            firstind = thisind
            for a in AList[thisind]:
                dists = [math.sqrt((a[1][0][0]-tubeSplits[k][-1][1][0][0])**2 + (a[1][0][1] - tubeSplits[k][-1][1][0][1])**2) for k in range(0,len(tubeSplits))]
                #if min(dists) < 400:
                tubeSplits[dists.index(min(dists))].append(a)
            
        
    
                
    #at the end of the tube
    if tubeSplits:

        #print("Tube", i,"is getting snipped.")
        thisA = [e[0] for e in L[i]]
        c = Counter(thisA)
        newrange = [a for a in thisA if c[a]<=1]
        #newrange=[0]
        #print(newrange)
     

        #Remove the old tube where it was split
        L[i] = [e for e in L[i] if (e[0] in newrange) or (e[0]<min(newrange))]
        
        if newrange[-1]-newrange[0] <30:
            
            for T in tubeSplits: #add each of the tubes from the splits
                RR = random.randint(0,255)
                G = random.randint(0,255)
                B = random.randint(0,255)
                for e in T:
                    e[3][0] = RR
                    e[3][1] = G
                    e[3][2] = B
                tubesToAdd.append(T)

        #print(therange)
        #L[i] = [e for e in L[i] if e[0] not in therange]
        #IF SOMETHING IS BROKEN IT'S THIS LINE RIGHT UP HERE^^^ 
        
        
       
            

L.extend(tubesToAdd)
    
newpaths = L
stage2tubes = newpaths
#------------------------------------------------------------------------------------------------------

L = deepcopy(newpaths)

#Sorting
for t in L:
    t.sort(key = lambda x: x[0])

temporalcutoff = 20
newpaths = [p for p in L if len(p)> temporalcutoff]        
janky = [p for p in newpaths if (p[1][0]-p[0][0] > temporalcutoff)]

#print(len(janky))
#print(len(newpaths))

for k in range(len(newpaths)):
    if ((newpaths[k][-1][0] - newpaths[k][round(len(newpaths[k])/2)][0]) > round(len(newpaths[k])/2)+50) or ( (newpaths[k][round(len(newpaths[k])/2)][0]) - newpaths[k][0][0] > round(len(newpaths[k])/2)+50) :
        janky.append(newpaths[k])
    
    #if (newpaths[k][-2][0]+ newpaths[k][-1][0])/2 > newpaths[k][-4][0] + 100 : #if there's a huge gap at end
        #newpaths[k] = newpaths[k][:-2]
        
newpaths = [p for p in newpaths if (p not in janky)]        
#Get the longest consecutive version of each tube --> make sure it's more than 20
nj = []
for janktube in janky:
    inds = [p[0] for p in janktube]  
    print(inds)
    stretches = []
    i=0
    j=0
   
    while i < len(inds)-1:
        startf = inds[i]
        nextf = inds[i+1]
       
        while (nextf-startf<=3) and j <len(inds)-1:
            #keep going
            startf = inds[j]
            nextf = inds[j+1]
            j=j+1
            
            
        stretch = [k for k in inds if k>= inds[i] and k<nextf]
        stretches.append(stretch)
        i=i+1
        j=i
        
    beststretch = max(stretches, key = lambda x: len(x)) 
    print(beststretch)
    print()
    newtube = [e for e in janktube if e[0] in beststretch]
    if len(newtube)> temporalcutoff:
        nj.append(newtube)
   
for t in nj:
    newpaths.append(t)

    
#for t in range(len(newpaths)):
#    for f in range(newpaths[t][0][0],newpaths[t][-1][0]):
#        here = [e for e in newpaths[t] if e[0]==f]
#        if len(here)>1:
#            here.sort(key = lambda x: -x[1][1][0])
#            bige = getBoxFromEllipse(here[0])
#            for i in range(1,len(here)):
#                lile = getBoxFromEllipse(here[i])
#                if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
#                    newpaths[t].remove(here[i])
            
    
stage3tubes = newpaths
#-----------------------------------------------------------------------------------------------------


L = deepcopy(newpaths)
tubes = []

recalcVelocity(L,5)

        
##Quick deletion of anything that has multiple ellipses on a frame... it's probably leftover from S2\n",
#for k in range(0,len(L)):
#    badframes=[]
#    for f in range(L[k][0][0], L[k][-1][0]):
#        flist = [e for e in L[k] if e[0] ==f]
#        sizeList = [(e[1][1][0]+e[1][1][1])/2 for e in flist]
#        if len(flist)>1 :
#            badframes.append(f)
#    L[k] = [e for e in L[k] if e[0] not in badframes]



##Quick separation of anything that has multiple ellipses on a frame... it's probably leftover from S2
#toAdd = []
#for k in range(len(L)):
#    newtube = []
#    alldoubles = [[e for e in L[k] if e[0]==f ] for f in range(L[k][0][0],L[k][-1][0]) if len([e for e in L[k] if e[0]==f])>=2]
#    for pair in alldoubles: #first remove all interior ellipses
#        pair.sort(key = lambda x: (x[1][1][0]))
#        lile = getBoxFromEllipse(pair[0])
#        bige = getBoxFromEllipse(pair[1]) 
#        if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
#                L[k].remove(pair[0])
#                
#    alldoubles = [[e for e in L[k] if e[0]==f ] for f in range(L[k][0][0],L[k][-1][0]) if len([e for e in L[k] if e[0]==f])>=2]
#    if alldoubles:
#        i=0
#        starte = alldoubles[0][0]
#        while i< len(alldoubles)-1:
#            pair = alldoubles[i]
#            pair.sort(key = lambda x: (starte[1][0][0]-x[1][0][0])**2 + (starte[1][0][1] - x[1][0][1])**2 )
#            starte = pair[0]
#            print("Going with",starte)
#            newtube.append(starte)
#            L[k].remove(starte)
#            i=i+1
#
#    if len(newtube)>10:
#        toAdd.append(newtube)
#if toAdd:        
#    L.extend(toAdd)
         
    

#Each slice should be [f, ((X,Y),(b,A),rot), [Vxavg,Vyavg], [R,G,B]]

for i in range(0,len(L)):
    slicestoAdd = []
    for f in range(int(L[i][0][0]), int(L[i][-1][0])):
        #chunk = [L[i].index(e) for e in L[i] if abs(e[0]-f) < 15]
        
        thisone = [e for e in newpaths[1] if e[0]==f]
        if not thisone: #if we don't have a slice at this frame
            thischunk = []
            a = 2
            while len(thischunk) < 3:
                thischunk = [L[i].index(e) for e in L[i] if abs(e[0]-f)<=a]
                a=a+2
                
            chunkX = [L[i][ind][1][0][0] for ind in thischunk]
            X = sum(chunkX)/len(chunkX)
            chunkY = [L[i][ind][1][0][1] for ind in thischunk]
            Y = sum(chunkY)/len(chunkY)
            chunksmallax = [L[i][ind][1][1][0] for ind in thischunk]
            smallax=max(chunksmallax)
            chunkbigax = [L[i][ind][1][1][1] for ind in thischunk]
            bigax=max(chunkbigax)
            chunkphi = [L[i][ind][1][2] for ind in thischunk]
            phi=max(chunkphi)
            chunkf = [L[i][ind][0] for ind in thischunk]
            Vx = (chunkX[-1] - chunkX[0])/(max(chunkf)+1-min(chunkf))
            Vy = (chunkY[-1] - chunkY[0])/(max(chunkf)+1-min(chunkf))
        
            thisslice = [f, ((X,Y),(smallax,bigax),phi), [Vx,Vy], L[i][0][3]]
            slicestoAdd.append(thisslice)
            
    L[i].extend(slicestoAdd)
            
tubes = L

#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one 
L = deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: (x[1][1][0]**2 + x[1][1][1]**2) + (x[2][0]**2+x[2][1]**2) ) 
            toRemove.extend(A[:-1])
            

    tubes.append([p for p in L[i] if p not in toRemove])
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
  

stage4tubes = tubes
print("End of S4")
#----------------------------------------------------------------------------------------------------


# Stage 5: Merging tubes! This first merge is a short term merge (lookahead = 5 frames)

#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
    
#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 10 #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]

lookahead = 5

M5 = getMerges(L,lookahead)
print(M5)
tubes = doMerge(L,M5)


#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)
    
    
stage5tubes = tubes
#----------------------------------------------------------------------------------------------------



# Event detection in preparation for the next stage(s)


#Putting into the familiar form. e = [x,y,a,b,phi,f,Vx,Vy,R,G,B, tubenum] 
#Prepping the data for event detection
tubeList=[]
L = deepcopy(tubes)
for tube in L:
    for e in tube:
        ellipse = [e[1][0][0], e[1][0][1], e[1][1][1]/2, e[1][1][0]/2, e[1][2], e[0], e[2][0], e[2][1], e[3][0], e[3][1], e[3][2],L.index(tube) ]
        tubeList.append(ellipse)

DATA = np.array(tubeList)


#This is code that I wrote a while back to detect "events"... when tubes get a little too close for comfort

sThresh = min(2*np.mean(AVGS), 100)
tThresh = 3

eventList = []
#Each event is a "collision" of two tubes t1 and t2 given by [f1,x1,y1,t1,f2,x2,y2,t2]

for f in range(tThresh,F):
    inTime = [ (row[5],row[0],row[1],row[-1]) for row in DATA if abs(row[5] - f) < tThresh ]
    inTimesizes = [(row[2]+row[3],row[-1]) for row in DATA if abs(row[5]-f) < tThresh]
    
    for comprow in inTime:
        notSameTube = np.array([row for row in inTime if row[-1] != comprow[-1]])
        nSTsizes = np.array([row for row in inTimesizes if row[-1] !=comprow[-1]])
        #print(nSTsizes)
        #print()
        
        if notSameTube.size!=0:
            checklocs = [ [ A[1], A[2]] for A in notSameTube] #locations
            thisloc = np.array([comprow[1], comprow[2]])
            #thissize = comprow[3]
            diff = np.linalg.norm((thisloc - checklocs),axis=1)
            sizecheck = min(nSTsizes[:,0])
            #print(diff)
            #print()

            closeIndex = [i for i,x in enumerate(diff) if x < sThresh]
            closeOnes = [notSameTube[i] for i in closeIndex]
            #print(closeOnes)

            for cL in closeOnes:
                CR = list(comprow)
                CR.extend(list(cL))
                eventList.append(CR)
                 

            
majorEventList = []
# Each major event combines the large groups of events into a smaller format [fi, ff, xi, xf, yi, yf, t1, t2]
# where i,f are initial and final and location is an avg of the two tubes

if eventList:
    datubes = np.sort([eventList[0][3], eventList[0][7]])
    startf = int(eventList[0][0])
    startloc = [np.mean((eventList[0][1],eventList[0][5])) , np.mean((eventList[0][2],eventList[0][6]))]

for i in range(len(eventList)):
    E = eventList[i]
    ts=np.sort([E[3], E[7]])
    

    if np.array_equal(ts,datubes):
        continue
    else:
        D=eventList[i-1]
        endf=int(D[0])
        endloc= [np.mean((D[1],D[5])) , np.mean((D[2],D[6]))]
        me = [startf, endf, startloc[0], endloc[0], startloc[1], endloc[1], int(datubes[0]), int(datubes[1])]
        #print(me)
        majorEventList.append(deepcopy(me))
        
        E = eventList[i]
        datubes=np.sort([E[3], E[7]])
        startf = int(E[0])
        startloc = [np.mean((E[1],E[5])) , np.mean((E[2],E[6]))]

Tlist = [ [me[-2],me[-1]] for me in majorEventList]
EventList = []

Tlistset = set(tuple(row) for row in Tlist)
for Tset in Tlistset:
    fList = []
    Tindices = [i for i,x in enumerate(Tlist) if tuple(x) ==Tset]
    for Ti in Tindices:    
        fList.append(majorEventList[Ti][0])
        fList.append(majorEventList[Ti][1])
    fmin = min(fList)
    fmax = max(fList)
    EventList.append([fmin, fmax, majorEventList[Tindices[0]][2], majorEventList[Tindices[-1]][2], majorEventList[Tindices[0]][4], majorEventList[Tindices[-1]][4], Tset[0], Tset[1] ])
    
        
eventlist = EventList
eventList = EventList
print(len(EventList))
for e in EventList:
    print(e)
        
L = deepcopy(tubes)
deltaList = []

for t in L:
    dxy= []
    dV = []
    dab= []
    for i in range(1,len(t)):
        this = t[i]
        old = t[i-1]
        dxy.append(math.sqrt((this[1][0][0]-old[1][0][0])**2 + (this[1][0][1]-old[1][0][1])**2))
        dV.append(math.sqrt((this[2][0]-old[2][0])**2 + (this[2][1]-old[2][1])**2))
        dab.append(math.sqrt((this[1][1][0]-old[1][1][0])**2 + (this[1][1][1]-old[1][1][1])**2))
    deltaList.append([dxy, dV, dab])

#----------------------------------------------------------------------------------------------------


# Stage 5.5: Makes sure that tubes are connected across the merge without gaps.

for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)    
    
for t in L:
    if (len(t) >= (t[-1][0] - t[0][0] + 1)):
        continue
        
    else:
        print("Gaps in tube", tubes.index(t))
        toAdd = []
        have = [e[0] for e in t]
        donthave = [num for num in range(int(t[0][0]), int(t[-1][0])) if num not in have]
        dlist = []
        for k, g in groupby(enumerate(donthave), lambda ix :ix[0]-ix[1]):
            dlist.append(list(map(itemgetter(1), g) ))
        for gap in dlist:
            preve = [e for e in t if e[0]==gap[0]-1][0]
            prevsize = (preve[1][1][0]+preve[1][1][1])/2
            nextegroup = [e for e in t if (e[0]>= gap[-1]) and (e[0]<=gap[-1]+10) ]
            nextegroup.sort(key = lambda x: (x[1][1][0] + x[1][1][1])/2 - prevsize)
            nexte = nextegroup[0]
            fgap = len(gap)+1
            for f in gap:
                framediff = f - (gap[0]-1)
                
                X = (nexte[1][0][0] - preve[1][0][0]) * (framediff/fgap) + preve[1][0][0]
                Y = (nexte[1][0][1] - preve[1][0][1]) * (framediff/fgap) + preve[1][0][1]
                smallax = (nexte[1][1][0] - preve[1][1][0]) * (framediff/fgap) + preve[1][1][0]
                bigax = (nexte[1][1][1] - preve[1][1][1]) * (framediff/fgap) + preve[1][1][1]
                phi = nexte[1][2]
                #phi = (nexte[1][2] - preve[1][2]) * (framediff/fgap) + preve[1][2]
                Vx = (X - preve[1][0][0])/framediff
                Vy = (Y - preve[1][0][1])/framediff
                RGB = preve[3]
                
                newe = [f, ((X,Y), (smallax,bigax), phi), [Vx,Vy], RGB]
                toAdd.append(newe)
          
        t.extend(toAdd)  
            

noEngulf = []
for t in L:
    
    innercount=0
    for e in t:
        lile = getBoxFromEllipse(e)
        esonframe = [ell for u in L for ell in u if ell[0]==e[0]]
        esonframe.remove(e)
        for ell in esonframe:
            bige = getBoxFromEllipse(ell) 
            if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
                innercount = innercount+1
                break
    if innercount/len(t) > .5:
        print("Tube ",L.index(t),"is an inner tube")
        noEngulf.append(L.index(t))            
            
tubes = L
for t in tubes:
    t.sort(key = lambda x: x[0])
    
stage55tubes = tubes    

#----------------------------------------------------------------------------------------------------
#Want to figure out which tubes in S5 are basically non moving so they can't be involved in S6
CPcount = []
S6scores = []
noEngulf = []
for t in stage55tubes:
    Xs = np.array([e[1][0][0] for e in t])
    Ys = np.array([e[1][0][1] for e in t])
    dX = np.diff(Xs, n=1)
    dY = np.diff(Ys, n=1)
    CPx = 0
    CPy = 0
    
    for i in range(1,len(t)-1):
        if (dX[i-1]<0) != (dX[i]<0):
            CPx = CPx+1
        if (dY[i-1]<0) != (dY[i]<0):
            CPy = CPy+1      
    CPcount.append([CPx, CPy])
   
    s1 = min(CPx,CPy)/len(t)
    Xdiff = abs(Xs[-1]-Xs[0])
    Ydiff = abs(Ys[-1]-Ys[0])
    s2 = (Xdiff+Ydiff)/2
    
    innercount=0
    for e in t:
        lile = getBoxFromEllipse(e)
        esonframe = [ell for u in stage55tubes for ell in u if ell[0]==e[0]]
        esonframe.remove(e)
        for ell in esonframe:
            bige = getBoxFromEllipse(ell) 
            if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
                innercount = innercount+1
                break
    if innercount/len(t) > .5:
        print("Tube ",stage55tubes.index(t),"is an inner tube")
        noEngulf.append(stage55tubes.index(t))
        
    
    print("Tube ", stage55tubes.index(t), [CPx, CPy],". Length = ",len(t),"Score1: ",s1, "Score2:",s2 )
    if s1> 0.22 or (s2<50 and len(t)<100):
        print("Tube", stage55tubes.index(t),"shouldn't engulf or be engulfed")
        noEngulf.append(stage55tubes.index(t))

        
print(noEngulf)

#Painting the "noEngulf" tubes white just to show them later
#for i in range(len(stage55tubes)):
#    if i in noEngulf:
#        for e in stage55tubes[i]:
#            e[3] = [255,255,255]

#3/7/21 changes made to S6 include adding the noEngulf thing to gatekeep
#Also added that if track runs into a separate tube, be sure to delete the orig tube ellipses  



# Stage 6: Detects and handles the collision where one tube "swallows" another by 
# implementing short term trackers on each (pre-collision) before reconnecting whenever possible.
# Used to be known as stage 8, but got moved up here in the process.

    

L = deepcopy(stage55tubes)
tracks = []
closeThresh = 30
steadyThresh = 10
hitList = []

for i in range(0,len(L)):
    if L[i]:
        lastf = L[i][-1][0]
        #print("Lastf", lastf)
        a = [e[-2] for e in EventList if i in e[-2:]]
        b = [e[-1] for e in EventList if i in e[-2:]]
        etubes = list(set(a+b))
        if etubes:
            etubes.remove(i)   #now we have a list of tubes to look at
            for t in etubes:
                #Find the start of this event...
                thisEvent = [e for e in EventList if (i in e[-2:]) and (t in e[-2:])][0]
                #print(thisEvent)
                eventStart = thisEvent[0]
                eventEnd = thisEvent[1]
                startf = L[t][0][0]
                dpos = np.mean(deltaList[t][0][lastf-startf-3:lastf-startf+3])
                dsize= np.mean(deltaList[t][2][lastf-startf-3:lastf-startf+3])
                if math.isnan(dpos) or math.isnan(dsize):
                    continue
                if (1.5*np.mean(deltaList[t][0]) < dpos) and (1.5*np.mean(deltaList[t][2]) < dsize) and (i not in noEngulf) and (t not in noEngulf) and (abs(eventStart-lastf)<=20 or abs(eventEnd-lastf)<=20):
                    print("We got a hit!")
                    print("First tube is ", i, "and swallowing tube is ", t)
                    print("Average position delta:", np.mean(deltaList[t][0]) )
                    print("Here it's: ",dpos)
                    print("Average size delta:", np.mean(deltaList[t][2]) )
                    print("Here it's ",dsize)
                    hitList.append(i)

                    track1 = []
                    track2 = []
                    efirst = [e for e in L[i] if (e[0]>=lastf-3) and (e[0] <=lastf)]
                    esec = [e for e in L[t] if (e[0]>=lastf-3) and (e[0] <=lastf)]
                    if not(efirst) or not(esec):
                        continue
                    efirst = efirst[0]
                    esec = esec[0]
                    tracker1 = cv2.TrackerCSRT_create()
                    tracker2 = cv2.TrackerCSRT_create()
                    bbox1 = getBoxFromEllipse(efirst)
                    bbox2 = getBoxFromEllipse(esec)
                    ok1 = tracker1.init(invid[int(lastf-3)], bbox1)
                    ok2 = tracker2.init(invid[int(lastf-3)], bbox2)
                    track1.append((int(lastf-3), bbox1))
                    track2.append((int(lastf-3), bbox2))
                    L[t] = [e for e in L[t] if e[0] <= lastf-3]

                    OutofBoundsFlags = [False, False]
                    NewTubeFlags = [False, False]
                    SteadyTrackFlags = [False, False]
                    track2Steady = False
                    f = lastf-3

                    while not(OutofBoundsFlags[0]) and not(NewTubeFlags[0]) and (f<F-1) :
                        f = f+1
                        r1= 9999
                        ok1, bbox1 = tracker1.update(invid[int(f)])
                        center1 = (bbox1[0]+bbox1[2]/2 , bbox1[1]+bbox1[3]/2)

                        if ok1:
                            track1.append((int(f), bbox1))
                            

                        #Check for NEW tube super close to the bbox (after about 20 frames)
                        if f > lastf+30:
                            for thistube in L:  
                                #if L.index(thistube)==t or L.index(thistube)==i: #not including the collision one
                                #    continue
                                Thise = [e for e in thistube if e[0] == f]
                                if Thise:
                                    thise = Thise[0]
                                    r1 = math.sqrt((thise[1][0][0] - center1[0])**2 + (thise[1][0][1] - center1[1])**2 ) 
                                if r1 < closeThresh:
                                    print("Tube ", L.index(thistube),"intersects at", thise)
                                    NewTubeFlags[0] = True
                                    newtubef = f
                                    print("Tube", L.index(thistube),"was",r1, "away from tube",i," at frame",newtubef)
                                    break

                        #Check for OOB        
                        if (center1[0] > W-30) or (bbox1[0]<30) or (center1[1]>H-5) or (bbox1[1]<0):
                            OutofBoundsFlags[0] = True
                            print("Went outta bounds.")
                        '''  
                        #Check for a super steady track.
                        if len(track1) > steadyThresh:
                            ssum = 0
                            for k in range(-steadyThresh,-1):
                                rt = math.sqrt((track1[k][1][0] - track1[k+1][1][0])**2 + (track1[k][1][1] - track1[k+1][1][1])**2)
                                ssum = ssum + rt
                            if ssum<steadyThresh:
                                SteadyTrackFlags[0] = True
                                print("Track is too steady. Probably done.")
                                print("Movement in the last 30 frames = ", ssum,"pixels")
                        '''
                    if len(track1) >= 25:
                        #See if the last thing on the track is similar to something about 30 frames ago...
                        #If so, delete any track box within a 10 pixel radiuss of that one
                        rt = math.sqrt((track1[-1][1][0] - track1[-20][1][0])**2 + (track1[-1][1][1] - track1[-20][1][1])**2)
                        if rt<2:
                            print("At least part of the track on", i, "was too steady to trust.")
                            savetr = track1[0]
                            steadyList = [tr for tr in track1 if math.sqrt((track1[-1][1][0] - tr[1][0])**2 + (track1[-1][1][1] - tr[1][1])**2) < 30]
                            track1 = [tr for tr in track1 if tr not in steadyList]
                            if not(track1):
                                track1 = [savetr]

                        
                    #Go thru track1 and turn boxes into ellipses and put them in the proper tube
                    toAdd = []
                    for tr in track1:
                        ell = [tr[0], getEllipseFromBox(tr[1], efirst[1][2]), efirst[2], efirst[3]]
                        toAdd.append(ell)
                    L[i].extend(toAdd)
                    #If NewTubeFlag, then everything in the new tube
                    if NewTubeFlags[0] and (newtubef<=track1[-1][0]):
                        print("THISTUBE" , L.index(thistube))
                        L[i] = [e for e in L[i] if e[0] < newtubef] 
                        toAdd = [e for e in thistube if e[0]>=newtubef]
                        L[L.index(thistube)] = [e for e in thistube if e[0]<=newtubef]
                        for ell in toAdd:
                            ell[3] = L[i][0][3]
                        L[i].extend(toAdd)



                    f = lastf-3
                    while not(OutofBoundsFlags[1]) and not(NewTubeFlags[1]) and (f<F-1):
                        f = f+1
                        r2 =9999
                        ok2, bbox2 = tracker2.update(invid[int(f)])
                        center2 = (bbox2[0]+bbox2[2]/2 , bbox2[1]+bbox2[3]/2)

                        if ok2:
                            track2.append((int(f), bbox2))

                        #Check for NEW tube super close to the bbox
                        if f > lastf+30:
                            for thistube in L:
                                #if L.index(thistube)==t or L.index(thistube)==i: #not including the collision one
                                #    continue
                                Thise = [e for e in thistube if e[0] == f]
                                if Thise:
                                    thise = Thise[0]
                                    r2 = math.sqrt((thise[1][0][0] - center2[0])**2 + (thise[1][0][1] - center2[1])**2 ) 
                                if r2 < closeThresh:
                                    print("Tube ", L.index(thistube),"intersects at", thise)
                                    NewTubeFlags[1] = True
                                    newtubef = f
                                    print("Tube", L.index(thistube),"was",r2,  "away from tube",t," at frame",newtubef)
                                    
                                    break

                        #Check for OOB
                        if (center2[0] > W-30) or (bbox2[0]<30) or (center2[1]>H-5) or (bbox2[1]<0):
                            OutofBoundsFlags[1] = True
                        '''
                        #Check for a super steady track.
                        if len(track2) > steadyThresh:
                            ssum = 0
                            for k in range(-steadyThresh,-1):
                                rt = math.sqrt((track2[k][1][0] - track2[k+1][1][0])**2 + (track2[k][1][1] - track2[k+1][1][1])**2)
                                ssum = ssum + rt
                            if ssum<steadyThresh:
                                SteadyTrackFlags[1] = True
                                print("Track is too steady. Probably done.")
                                print("Movement in the last 30 frames = ", ssum,"pixels")
                        ''' 
                    if len(track2)>=25:       
                        #See if the last thing on the track is similar to something about 30 frames ago...
                        #If so, delete any track box within a 10 pixel radiuss of that one
                        rt = math.sqrt((track2[-1][1][0] - track2[-20][1][0])**2 + (track2[-1][1][1] - track2[-20][1][1])**2)
                        if rt<2:
                            print("At least part of the track on", t, "was too steady to trust.")
                            print("Using the old one.")
                            savetr = track2[0]
                            steadyList = [tr for tr in track2 if math.sqrt((track2[-1][1][0] - tr[1][0])**2 + (track2[-1][1][1] - tr[1][1])**2) < 30]
                            track2 = [tr for tr in track2 if tr not in steadyList]
                            track2Steady = True
                            if not(track2):
                                track2 = [savetr]

                    #Go thru track2 and turn boxes into ellipses and put them in the proper tube
                    toAdd = []
                    #print(track2)
                    if track2Steady:
                        #print([t[0] for t in track2])
                        L[t] = []
                        for tr in track2:
                            ell = [tr[0], getEllipseFromBox(tr[1], esec[1][2]), esec[2], esec[3]]
                            L[t].append(ell)
                        
                        L[t].extend( [e for e in stage55tubes[t] if e[0] >= track2[-1][0]])
                    else:
                        for tr in track2:
                            ell = [tr[0], getEllipseFromBox(tr[1], esec[1][2]), esec[2], esec[3]]
                            toAdd.append(ell)
                        L[t].extend(toAdd)
                        #If NewTubeFlag, then everything in the new tube
                        if NewTubeFlags[1] and (newtubef<=track2[-1][0]):
                            print("THISTUBE" , L.index(thistube))
                            L[t] = [e for e in L[t] if e[0] < newtubef] 
                            toAdd = [e for e in thistube if e[0]>=newtubef]
                            L[L.index(thistube)] = [e for e in thistube if e[0]<=newtubef] 
                            for ell in toAdd:
                                ell[3] = L[t][0][3]
                            L[t].extend(toAdd)



                    tracks.append(track1)
                    tracks.append(track2)

L = [tube for tube in L if tube!=[]]
tubes = L

#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 3  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]

#if the velocity puts you OOB soon, you out. #JK this does nothing now but output for reading
for h in hitList:
    startf = L[h][0][0]
    for e in L[h]:
        predX = e[1][0][0] + e[1][1][0]*3
        predY = e[1][0][1] + e[1][1][1]*3
        if (predX > W or predX<0) and(e[0]-startf >30):
            print(h,"is messed up horiz at frame", e[0])
            L[h] = [ell for ell in L[h] if ell[0]<=e[0]]
            break

tubes = L

print("Done.")
print("End of S6")

stage6tubes = tubes
#putTracksOnVid(tracks,vidbase+'_outstage5nums.mp4',vidbase+'_outstage6tracksUGLY.mp4')
#putTubesOnVid(tubes, vidbase+'_outstage6UGLY.mp4')


#---------------------------------------------------------------------------------------------


#STAGE 9---------this block is kinda long

# Using DBSCAN on each tube 

L=deepcopy(tubes)



labelList = []
for t in L:
    XforDB = np.array([[e[0],e[1][0][0], e[1][0][1]] for e in t])
    DBout = DBSCAN(eps=0.6*np.mean(AVGS), min_samples = 10).fit(XforDB)
    labelList.append(DBout.labels_)
    
#Populated labelList... now FIND OUT WHICH HAVE SUBTUBES AND PLOT THEM
#Thinking len(set()) > 2... but then the -1 thing smh idk
haveSubs = []
haveGlides=[]
haveSteadys=[]
for t in range(0,len(labelList)):
    labs = set(labelList[t])
    if -1 in labs:
        labs.remove(-1)
    if len(labs)>=2:
        haveSubs.append(t)
  

print(haveSubs)


#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 3  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]


# Recognize glides and steadys within a tube (and maybe split)
tubestoAdd=[]
for i in range(0,len(L)):
    scount=0
    gcount=0
    j=0
    slist=[]
    glist=[]
    while j+1 < len(L[i]):
        e= L[i][j]
        startf=e[0]
        
        if min([abs(a) for a in e[2]]) < 0.2 and max([abs(a) for a in e[2]])<0.5 :
            scount=scount+1
        else:
            scount=0
        
        if abs(e[2][0]-L[i][j-1][2][0])<0.1 and abs(e[2][1]-L[i][j-1][2][1])<0.1 :
            gcount=gcount+1
        else:
            gcount=0
            
        if scount>30:
            slist.append(startf)
        if gcount>30:
            #print("Gliding at tube", i)
            glist.append(startf)   
        j=j+1
        
    #print("Tube", i)
    #print(slist)
    #print(glist)
    if (len(slist)>3) and (len(glist)<len(slist)/2): #steady and not gliding...
        # Make a split. Add to haveSubs. Change labelList
        A = [e for e in L[i] if (e[0]<min(slist)-30)]
        B = [e for e in L[i] if (e[0]>min(slist)-30) and (e[0]<=max(slist))]
        C = [e for e in L[i] if (e[0]>max(slist))]
        haveSteadys.append(i)
        Ai = [L[i].index(e) for e in A]
        Bi = [L[i].index(e) for e in B]
        Ci = [L[i].index(e) for e in C]
        for a in Ai:
            labelList[i][a]=0
        for b in Bi:
            labelList[i][b]=1
        for c in Ci:
            labelList[i][c]=2
    
    if (len(glist)>3) and (len(slist)<len(glist)/2): #gliding, not steady...
        # Make a split. Add to haveSubs. Change labelList
        A = [e for e in L[i] if (e[0]<min(glist)-30)]
        B = [e for e in L[i] if (e[0]>min(glist)-30) and (e[0]<=max(glist))]
        C = [e for e in L[i] if (e[0]>max(glist))]
        haveGlides.append(i)
        Ai = [L[i].index(e) for e in A]
        Bi = [L[i].index(e) for e in B]
        Ci = [L[i].index(e) for e in C]
        #L[i] = [e for e in L[i] if (e in A) or (e in C)] #no glide
        #for a in Ai:
        #    labelList[i][a]=0
        #for b in Bi:
        #    labelList[i][b]=2
        #for c in Ci:
        #    labelList[i][c]=1
        #labelList[i] = [labelList[i][ind] for ind in range(0,len(labelList[i])) if (ind in Ai) or (ind in Ci) ]

print()
print("Has subtubes:", haveSubs)
print("Has glides:" ,haveGlides)
print("Has steadys:", haveSteadys)
    
    
#Want to remove any subtube (and anything after it) that is steady
#Only considering things with subtubes (SECOND TRY)

hasStuff = list(set(haveSteadys+haveSubs))

for h in hasStuff:
    STlabels = labelList[h]
    T = L[h]
    for l in set(STlabels): #for each subtube
        ST = [T[ind] for ind in range(0,len(STlabels)) if STlabels[ind]==l] #actual subtube
        steadycount=0
        count=0
        for e in ST:
            count=count+1
            if abs(e[2][0])<0.1 and abs(e[2][1])<0.1:
                steadycount=steadycount+1
        perc = steadycount/count
        print(perc, "on tube",h," subtube", l)
        
        
        if perc > 0.5 and l!=-1:
            firstind = list(STlabels).index(l)
            print("Percent steady=",perc,"on tube",h," subtube", l, "starting at", firstind)
            #labelList[h] = [STlabels[ind] for ind in range(0,len(STlabels)) if STlabels[ind]!=l]
            #DBtubes[h] = [T[ind] for ind in range(0,len(T)) if STlabels[ind]!=l]
            labelList[h] = STlabels[0:firstind]
            L[h] = T[0:firstind]
            break
                        

#print(len(labelList))
#print(len(L))
#L = deepcopy(DBtubes)
            
#Remove empty tubes... clear them from haveSubs and labelList also
toDel = [i for i in range(0,len(L)) if len(L[i])==0]
toDel.sort(reverse=True)
print("Whole tubes to delete:", toDel)
for d in toDel:
    haveSubs = [h-1  if h>d else h for h in haveSubs]
    if d in haveSubs:
        haveSubs.remove(d)
    haveGlides = [h-1 if h>d else h for h in haveGlides]
    if d in haveGlides:
        haveGlides.remove(d)
    
DBtubes = [t for t in L if L.index(t) not in toDel]
labelList = [labelList[i] for i in range(0,len(labelList)) if i not in toDel]

print(haveSubs)
#putSomeTubesOnVid(L, vidbase+'_outstage9numsNoSteadys.mp4',which = haveSubs )


#DBtubes = deepcopy(L)
for t in range(0,len(DBtubes)):
    RGB0 = DBtubes[t][0][3]
    RGB1 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB2 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB3 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB4 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB5 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB6 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB7 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB8 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGBM1 = [0, 0, 255]
    
    for i in range(0,len(DBtubes[t])):
        if labelList[t][i]==0:
            DBtubes[t][i][3] = RGB0
        elif labelList[t][i]==1:
            DBtubes[t][i][3] = RGB1
        elif labelList[t][i]==2:
            DBtubes[t][i][3] = RGB2
        elif labelList[t][i]==3:
            DBtubes[t][i][3] = RGB3
        elif labelList[t][i]==4:
            DBtubes[t][i][3] = RGB4
        elif labelList[t][i]==5:
            DBtubes[t][i][3] = RGB5
        elif labelList[t][i]==6:
            DBtubes[t][i][3] = RGB6
        elif labelList[t][i]==7:
            DBtubes[t][i][3] = RGB7
        elif labelList[t][i]==8:
            DBtubes[t][i][3] = RGB8
        else:
            DBtubes[t][i][3] = RGBM1
    
    
#putSomeTubesOnVid(DBtubes, vidbase+'_outstage9numsDBonly.mp4',which = haveSubs )
#putTubesOnVid(DBtubes, vidbase+'_outstage9numsDBsubtubes.mp4', True)


#Now we want to get a motion score for each subtube connection WITHIN TUBE

subtubes = []

for h in haveSubs:
    T = deepcopy(DBtubes[h])
    labels = labelList[h]
    splittube=[]
    A = set(labels)
    if -1 in A:
        A.remove(-1)
    for l in A:
        st=[T[ind] for ind in range(0,len(labels)) if labels[ind]==l]
        if len(st)>0:
            splittube.append(st)
    
   
    splittube.sort(key = lambda x: x[0][0])
    subtubes.append(splittube)
    #SUBTUBES NEED TO BE IN FRAME ORDER 
    
#Populated subtubes [[tube in sections], [another one], [[like],[this]]   ]
#Subtubes var goes with haveSubs... same length and haveSubs is the index for the matching tube
#SubTubes (capital T) just means i removed the "outlier" ones from the var

#subTubes = [[st for st in splitT if len(st)>=10] for splitT in subtubes]
subTubes = subtubes
stsizes = [[len(st) for st in splitT] for splitT in subTubes]
stscores= [[0 for st in splitT ] for splitT in subTubes]
print(stsizes)
print(stscores)


for st in stsizes:
    if len(st)>0:
        mainSTindex = np.argmax(st)
        thisindex = mainSTindex
        stscores[stsizes.index(st)][mainSTindex] = -1
        tubenum = haveSubs[stsizes.index(st)]
        #IF TUBENUM IS IN HAVEGLIDES... DON'T DO ANYTHING FOR MOTION. DO SOMETHING WITH STSCORES TO FORCE SPLIT
        if tubenum in haveGlides:
            #stscores[stsizes.index(st)] = [-1, -0.5] #force a split with negative num and skip motion stuff
            #subTubes[stsizes.index(st)] = [[],[]]
            #subTubes[stsizes.index(st)][0] = [DBtubes[tubenum][ind] for ind in range(0,len(DBtubes[tubenum])) if labelList[tubenum][ind]==0 ]
            #subTubes[stsizes.index(st)][1] = [DBtubes[tubenum][ind] for ind in range(0,len(DBtubes[tubenum])) if labelList[tubenum][ind]==1 ]
            continue

        print()
        while len(st)-1 > thisindex:  #if there are more subtubes after
            #Forward motion score
            print("Going forward on tube ",tubenum,"...")
            thisST = subTubes[stsizes.index(st)][thisindex]
            nextST = subTubes[stsizes.index(st)][thisindex+1]
            R = (thisST[-1][1][1][0]+thisST[-1][1][1][1])/2
            predf = nextST[0][0]
            predX = thisST[-1][2][0]*(thisST[-1][0]-predf) + thisST[-1][1][0][0]
            predY = thisST[-1][2][1]*(thisST[-1][0]-predf) + thisST[-1][1][0][1]
            diffX = abs(nextST[0][1][0][0] - predX)
            diffY = abs(nextST[0][1][0][1] - predY)
            Xscore = math.exp(-(diffX/R))
            Yscore = math.exp(-(diffY/R))
            print("X score is:", Xscore)
            print("Y score is:", Yscore)

            #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
            thisT = np.array([e[0] for e in thisST])
            thisX = np.array([e[1][0][0] for e in thisST])
            thisY = np.array([e[1][0][1] for e in thisST])
            nextT = np.array([e[0] for e in nextST])
            nextX = np.array([e[1][0][0] for e in nextST])
            nextY = np.array([e[1][0][1] for e in nextST])

            thisXofT = np.polyfit(thisT,thisX,1)
            thisYofT = np.polyfit(thisT,thisY,1)
            nextXofT = np.polyfit(nextT,nextX,1)
            nextYofT = np.polyfit(nextT,nextY,1)

            #print(thisXofT,nextXofT)
            #print(thisYofT,nextYofT)
            Xangle = np.arctan(abs((nextXofT[0]-thisXofT[0]) / (1 + nextXofT[0]*thisXofT[0]))) 
            Xcos = np.cos(Xangle)
            Yangle = np.arctan(abs((nextYofT[0]-thisYofT[0]) / (1 + nextYofT[0]*thisYofT[0]))) 
            Ycos = np.cos(Yangle)
            print("X cosine score", Xcos)
            print("Y cosine score", Ycos)

            finalscore = (max(Xscore,Yscore) + (Xcos+Ycos)/2)/2
            if len(nextST)<=20:
                finalscore = 1
            stscores[stsizes.index(st)][thisindex+1] = finalscore
            print("FINAL:", finalscore)



            thisindex=thisindex+1

        thisindex = mainSTindex    
        while thisindex>0:  #if there are more subtubes before
            #Backwards motion score
            print("Going backward on tube", tubenum ,"...")
            thisST = subTubes[stsizes.index(st)][thisindex]
            lastST = subTubes[stsizes.index(st)][thisindex-1]
            R = (thisST[0][1][1][0]+thisST[0][1][1][1])/2
            predf = lastST[-1][0]
            predX = thisST[0][1][0][0] - thisST[0][2][0]*abs(thisST[0][0]-predf) 
            predY = thisST[0][1][0][1] - thisST[0][2][1]*abs(thisST[0][0]-predf) 
            diffX = abs(lastST[-1][1][0][0] - predX)
            diffY = abs(lastST[-1][1][0][1] - predY)
            Xscore = math.exp(-(diffX/R))
            Yscore = math.exp(-(diffY/R))
            print("X score is:", Xscore)
            print("Y score is:", Yscore)


            #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
            thisT = np.array([e[0] for e in thisST])
            thisX = np.array([e[1][0][0] for e in thisST])
            thisY = np.array([e[1][0][1] for e in thisST])
            lastT = np.array([e[0] for e in lastST])
            lastX = np.array([e[1][0][0] for e in lastST])
            lastY = np.array([e[1][0][1] for e in lastST])

            thisXofT = np.polyfit(thisT,thisX,1)
            thisYofT = np.polyfit(thisT,thisY,1)
            lastXofT = np.polyfit(lastT,lastX,1)
            lastYofT = np.polyfit(lastT,lastY,1)

            #print(lastXofT,thisXofT)
            #print(lastYofT,thisYofT)
            Xangle = np.arctan(abs((lastXofT[0]-thisXofT[0]) / (1 + lastXofT[0]*thisXofT[0]))) 
            Xcos = np.cos(Xangle)
            Yangle = np.arctan(abs((lastYofT[0]-thisYofT[0]) / (1 + lastYofT[0]*thisYofT[0]))) 
            Ycos = np.cos(Yangle)
            print("X cosine score", Xcos)
            print("Y cosine score", Ycos)

            finalscore = (max(Xscore,Yscore) + (Xcos+Ycos)/2)/2
            if len(lastST)<=20:
                finalscore=1
            stscores[stsizes.index(st)][thisindex-1] = finalscore
            print("FINAL:", finalscore)

            thisindex = thisindex-1

        #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
        #THEN COMBINE THAT WITH THE BEST X/YSCORE FROM ABOVE TO GET FINAL MOTION SCORE
print(haveSubs)       
print(stscores)
#stsizes = [s for s in stsizes if len(s)>0]
#stscores = [s for s in stscores if len(s)>0] #no empties
#subTubes = [s for s in subTubes if len(s)>0]


#Putting subtubes back so they can be viewed (DBtubes2)

subInds = []
alpha = 0.8

for i in range(0,len(subTubes)): #for each split tube
    scores = stscores[i]
    startind = scores.index(-1)
    indLists = [[startind]]
    thisind = startind
    while len(scores)-1 > thisind:  #Forward Loop
        if scores[thisind+1] > alpha:
            A = [l for l in indLists if thisind in l][0] #the one with thisind
            A.append(thisind+1)
        else:
            indLists.append([thisind+1])
        thisind=thisind+1

    thisind = startind    
    while thisind>0: #Backward Loop
        if scores[thisind-1] > alpha:
            A = [l for l in indLists if thisind in l][0] #the one with thisind
            A.append(thisind-1)
        else:
            indLists.append([thisind-1])
        thisind = thisind-1

    subInds.append(indLists)

print(subInds)
newsubTubes = []
for st in range(0,len(subInds)):
    splitT = []
    for group in subInds[st]:
        #thisST = subTubes[st][thisindex]
        A = []
        for g in group:
            thisST = subTubes[st][g]
            A.extend(thisST)
        splitT.append(A)

    newsubTubes.append(splitT)
    
    
#Change color based on subtube    
DBtubes2 = []
for i in range(0,len(DBtubes)):
    if i not in haveSubs:
        DBtubes2.append(DBtubes[i])
    else:
        T = []
        RGB0 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB1 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB2 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB3 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        stindex = haveSubs.index(i)
        for coll in newsubTubes[stindex]: #for each collection of ellipses
            groupnum = newsubTubes[stindex].index(coll)
            for ell in coll:
                if groupnum==0:
                    ell[3] = RGB0
                elif groupnum==1:
                    ell[3] = RGB1
                elif groupnum==2:
                    ell[3] = RGB2
                else:
                    ell[3] = RGB3
            T.extend(coll)
        DBtubes2.append(T)
            
for h in haveSubs:
    grouping = subInds[haveSubs.index(h)]
    print(grouping)
    for g in grouping:
        labelList[h] = [x if (x==min(g) or x not in g) else min(g) for x in labelList[h] ]
                
 
 # Just finished doing the motion consideration for subtubes
    #Spatial consideration to come....

# Playing with SURF features to see if they can be compared...
# Want to check features within the ellipse at the end of subtube and start of the next... for now

#For now it'll be two frames from the same tube

for l in range(0,len(labelList)):
    if not (len(DBtubes2[l]) == len(labelList[l])):
        print("weird",l)
        A = [DBtubes[l][ind] for ind in range(0,len(labelList[l])) if labelList[l][ind]==-1 ]
        DBtubes2[l].extend(A)
        
    

for t in DBtubes2:
    t.sort(key = lambda x: x[0])
for s in subInds:
    s.sort(key = lambda x: min(x))

subInds2=deepcopy(subInds)
#surf = cv2.SURF_create()
orb = cv2.ORB_create()

feats = []

for s in range(0,len(subInds)):
    if len(subInds[s])>1:
        tubenum = haveSubs[s]
        splitT = []
        print("Tube:", tubenum)
        for st in subInds[s]: #for each subtube
            ST = [DBtubes2[tubenum][i] for i in range(0,len(labelList[tubenum])) if labelList[tubenum][i] in st]
            splitT.append(ST)
            DaMatches = []

        
        #for each thing in splitT if i+1 < len
        for i in range(0,len(splitT)-1):
            laste = splitT[i][-1]
            nexte = splitT[i+1][0]
            bbox = getBoxFromEllipse(laste)
            bbox2 = getBoxFromEllipse(nexte)
            img = V[int(laste[0])][bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
            img2 = V[int(nexte[0])][bbox2[1]:bbox2[1]+bbox2[3] , bbox2[0]:bbox2[0]+bbox2[2]]
            KP1,desc1 = orb.detectAndCompute(img,None)
            KP2,desc2 = orb.detectAndCompute(img2,None)
            print(len(KP1), len(KP2))
           
            
            if min(len(KP1),len(KP2)) > 5:
                # Brute Force Matching
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                #bf = cv2.BFMatcher(cv2.NORM_HAMMING)
                matches = bf.match(desc1, desc2)

                print("Matches",len(matches))
                spatialscore = len(matches)/ min(len(KP1),len(KP2))
                print(spatialscore)
                if spatialscore> 0.5:
                    print("MATCH.")
                    print()
                    A = deepcopy(subInds[s])
                    A[i].extend(A[i+1])
                    del A[i+1]
                    subInds2[s]=A
                    
                    
                matches = sorted(matches, key = lambda x:x.distance)
                DaMatches.append(matches)
                # Draw first 20 matches.
                img3 = cv2.drawMatches(img,KP1,img2,KP2,matches[:5],None, flags=2)

                #cv2.imshow("img3",img3)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                
            else:
                #you can't pull together 5 features
                print("No match.")
                print()
            
# subInds2 is the var that tells us the splits when there's a subtube.

tubesToAdd = []
for i in range(0,len(DBtubes2)):
    if i in haveSubs:
        splits = subInds2[haveSubs.index(i)]
        if len(splits)<=1:
            #Ya Good.
            print("no subtubes here..")
        else:
            toDel=[]
            for s in range(1,len(splits)):
                ST = [DBtubes2[i][ind] for ind in range(0,len(labelList[i])) if labelList[i][ind] in splits[s] ]
                tubesToAdd.append(ST)
                toDel.append([e for e in DBtubes2[i] if e not in ST])
            DBtubes2[i]=[e for e in DBtubes[i] if e not in toDel]
        
        
DBtubes2.extend(tubesToAdd)
#Fix color
for t in DBtubes2:
    RGB = t[0][3]
    for e in t:
        e[3] = RGB

# No ellipse can be in multiple tubes... fix.
L = deepcopy(DBtubes2)
for i in range(0, len(L)):
    for e in L[i]:
        A = [ell[1] for j in range(i,len(L)) for ell in L[j] if (ell[1]==e[1]) and(j!=i)]
        if A:
            #print("Tube",i,"and",j,"share this ellipse.")
            #print(A)
            L[i] = [e for e in L[i] if e[1] not in A]
DBtubes2=L
    

cv2.destroyAllWindows()
#-----------------------------------------------------------------------------------------------------------------
#Cleaning up


tubes = deepcopy(DBtubes2)
tubes = [tu for tu in tubes if len(tu)!=0]
toDel = []
hasgap = []
for t in tubes:
    #if len(t)<30:
    #    print("Tube ",tubes.index(t), "is too short")
    #    toDel.append(t)

    #Check for "inner" tubes and delete
    innercount=0
    for e in t:
        lile = getBoxFromEllipse(e)
        esonframe = [ell for u in tubes for ell in u if ell[0]==e[0]]
        esonframe.remove(e)
        for ell in esonframe:
            bige = getBoxFromEllipse(ell) 
            if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
                innercount = innercount+1
                break
    if innercount/len(t) > .5:
        print("Tube ",tubes.index(t),"is an inner tube")
        toDel.append(tubes.index(t))
        
    #Check for tube gaps, delete anything after gap    
    if abs(t[-1][0]-t[0][0] -len(t))>1:
        lastf = 0
        print("Tube ", tubes.index(t), "was cleaned up.")
        hasgap.append(tubes.index(t))

toCombine = []
for h in hasgap:
    for i in hasgap:
        #if h and i fit together perfectly
        if h!=i:
            hframes = [e[0] for e in tubes[h]]
            iframes = [e[0] for e in tubes[i]]
            comb = hframes+iframes
            comb.sort()
            #print(len(comb))
            if sum([(f in comb) for f in range(int(comb[0]),int(comb[-1]))]) ==len(comb)-1:
                if [i,h] not in toCombine and [h,i] not in toCombine:
                    print(h,"and",i,"fit together soooo...")
                    #toCombine.append([h,i])

supertube = []
for c in toCombine:
    supertube = tubes[c[0]] + tubes[c[1]]
    supertube.sort(key=lambda x: x[0])
    RGB = supertube[0][3]
    for e in supertube:
        e[3] = RGB
    toDel.append(c[0])
    toDel.append(c[1])
tubes.append(supertube)            

        
for t in tubes:
    if len(t)<12:
        print("Tube ",tubes.index(t), "is too short")
        toDel.append(tubes.index(t))

tubes = [tubes[i] for i in range(0,len(tubes)) if i not in toDel]


noMerge = []
maybeMerge=[]
#-----------------------------------------------------------------------------------------------------------------

#Maybe a preliminary merge where anything steady is linked to the next thing
# at that location (given a small enough gap in time)
recalcVelocity(tubes,5)
firstMerges = []
tracks = []
for t in tubes:
    lastfew = t[-15:-5]
    Xend = sum([e[1][0][0] for e in lastfew])/10
    Yend = sum([e[1][0][1] for e in lastfew])/10
    checkvel = [abs(e[2][0])<1 and abs(e[2][1])<1 for e in lastfew]
    checkell1 = t[-10]
    
    for k in tubes:
        checkell2 = k[10]
        diff = math.sqrt((checkell2[1][0][0]-Xend)**2 + (checkell2[1][0][1]-Yend)**2)
        if diff < min(checkell1[1][1][1],checkell2[1][1][1]) and k[0][0]>t[-1][0] and k[0][0]-t[-1][0]<240 and sum(checkvel)>8 :
            #print(tubes.index(t),"goes into",tubes.index(k))
            firstMerges.append([tubes.index(t),tubes.index(k)])

sectubes = [m[0] for m in firstMerges]
hasmany = [t for t in range(0,len(tubes)) if sectubes.count(t)>1]
toAdd = []

for h in hasmany:
    ms = [m for m in firstMerges if m[0]==h]
    othertubes = [m[1] for m in ms]
    #Check for zero overlap
    otherframes = [e[0] for t in othertubes for e in tubes[t]]
    otherframes.extend([e[0] for e in tubes[h]])
    if len(otherframes)==len(set(otherframes)):
        othertubes.append(h)
        othertubes.sort(key = lambda x: tubes[x][0][0])
        toAdd.append(othertubes)
        #print(otherframes)
    else:
        if len(othertubes)==2:
            othertubes.sort(key = lambda x: len(tubes[x]))
            othertubes=othertubes[1]
            toAdd.append([h,othertubes])
        else: #we got a clusterfuck going on...
            othertubes.append(h)
            othertubes.sort(key = lambda x: tubes[x][0][0])
            toAdd.append(othertubes)

firstMerges = [m for m in firstMerges if m[0] not in hasmany]
firstMerges.extend(toAdd)
toAdd=[]
toDel = []
print()
print(firstMerges)
for f in firstMerges:
    for g in firstMerges:
        if set(f).issubset(set(g)) and f!=g:
            toDel.append(f)
            
firstMerges = [f for f in firstMerges if f not in toDel]  
toDel=[]
toAdd = []
for f in firstMerges:
    #link start to next
    tolink = f[0]
    for g in firstMerges:
        if tolink==g[-1]:
            firstMerges.append(g[:-1]+f)
            toDel.append(f)
            
print()
print(firstMerges)
for f in firstMerges:
    for g in firstMerges:
        if set(f).issubset(set(g)) and f!=g:
            toDel.append(f)
firstMerges = [f for f in firstMerges if f not in toDel]            
firstMerges = [list(f) for f in set(tuple(row) for row in firstMerges)]
            
            
print()
print(firstMerges)

#Change the colors
for m in firstMerges:
    RGB = tubes[m[0]][0][3]
    for tu in m:
        for e in tubes[tu]:
            e[3] = RGB

#Make the tracks
for m in firstMerges:
    theframes = [[k,e[0]] for k in m for e in tubes[k]]
    therealframes= [e[0] for k in m for e in tubes[k]]
    theframes.sort(key= lambda x:x[1])
    #print(theframes)
    for f in range(int(theframes[0][1]),int(theframes[-1][1])):
        if f in therealframes:
            lastgood=theframes[therealframes.index(f)][0]
        else:
            track = [f, getBoxFromEllipse(tubes[lastgood][-10])]
            tracks.append(track)
            
            
#LINK TUBES BASED ON THAT FIRST MERGE
toDel = []
A = 10
colorList = []
for m in firstMerges:
    supertube = []
    checkpts = [tubes[t][A][0]  for t in m] + [tubes[t][-A][0]  for t in m]
    checkpts.sort()
    for t in m:
        toDel.append(t)
        supertube.extend(tubes[t])
    supertube.sort(key = lambda x: x[0])
    
    
    
    thefinal = supertube[-1][0]
    for thisf in range(int(supertube[0][0]), int(supertube[-1][0])):
        ishere = [e for e in supertube if e[0]==thisf]
        if ishere:
            continue
        else:
            low = thisf
            high = thisf
            while low not in checkpts and low>supertube[0][0]:
                low=low-1
            #print("The last", supertube[-1][0])
            while (high not in checkpts) and high<thefinal :
                high=high+1
                
            fgap = high-low
            laste = [e for e in supertube if e[0]==low][0]
            nexte = [e for e in supertube if e[0]==high][0]
            
            X = laste[1][0][0] + (nexte[1][0][0]-laste[1][0][0])*((thisf-low)/fgap)
            Y = laste[1][0][1] + (nexte[1][0][1]-laste[1][0][1])*((thisf-low)/fgap)
            smallax = laste[1][1][0] + (nexte[1][1][0]-laste[1][1][0])*((thisf-low)/fgap)
            bigax = laste[1][1][1] + (nexte[1][1][1]-laste[1][1][1])*((thisf-low)/fgap)
            phi = laste[1][2]
            Vx = laste[2][0] + (nexte[2][0]-laste[2][0])*((thisf-low)/fgap)
            Vy = laste[2][1] + (nexte[2][1]-laste[2][1])*((thisf-low)/fgap)
        
            thisslice = [thisf, ((X,Y),(smallax,bigax),phi), [Vx,Vy], RGB]
            supertube.append(thisslice)
            
    #do supertube colors
    RGB = tubes[m[0]][0][3]
    for e in supertube:
        e[3] = RGB
    
    #Add supertube. subtract old tubes
    tubes.append(supertube)
    colorList.append(RGB)
    
tubes = [T for T in tubes if tubes.index(T) not in toDel]
#-----------------------------------------------------------------------------------------------------------------



#------------------------------Merge suggestions----------------------------

        
#tubes = deepcopy(DBtubes2)
A = min(20, min([len(t) for t in tubes])-1 )
nT = len(tubes)
Merges = []



for t1 in range(0,nT):

    print("Observing Tube: ",t1)
    scorecard = np.empty([nT,9])
    #cantMerge = deepcopy(noMerge)
    cantMerge = []
    T1 = tubes[t1]
    T1cantmerge = False

    if t1 in noMerge:
        T1cantmerge = True
        print(noMerge)
        print(t1, "shoudn't merge at all.")

    #See if T1 goes outta bounds (unmergeable)
    #maxR = max([(e[1][1][1]+e[1][1][0])/2 for e in T1])
    edges = findEdges(V)
    maxmaj = max([e[1][1][1] for e in T1])
    avgVx= sum([e[2][0] for e in T1])/len(T1)
    if (T1[-A][1][1][0] + T1[-A][1][1][1])/2 <350:
        highXs = [e[1][0][0] + e[1][1][0]  for e in T1]
        lowXs = [e[1][0][0] - e[1][1][0] for e in T1]
        highYs = [e[1][0][1] + 0.5*e[1][1][0] for e in T1]
        lowYs = [e[1][0][1] - 0.5*e[1][1][0] for e in T1]
    else:
        highXs = [e[1][0][0]   for e in T1]
        lowXs = [e[1][0][0]  for e in T1]
        highYs = [e[1][0][1]  for e in T1]
        lowYs = [e[1][0][1]  for e in T1]
        
    tailend = round(len(highXs)*0.9)
    if max(highXs[tailend:])>edges[1]  or min(lowXs[tailend:])<edges[0]  or max(highYs[tailend:])>edges[3] or min(lowYs[tailend:])<edges[2]:
        print("This tube goes outta bounds, don't merge it.")
        T1cantmerge = True

    if T1cantmerge:
        Merges.append([t1,-1])
        continue



    #Fill out the scorecard for each eligible tube
    for k in range(0,nT):
        T2 = tubes[k]
        #print("Tube ",k)
        #Check if one is contained by the other, if so can't merge
        if ((T2[0][0]>T1[0][0]) and (T2[-1][0]<T1[-1][0])) or (T2[0][0]-T1[-1][0]>300):
            cantMerge.append(k)
            scorecard[k,:] = [0,0,0,0,0,0,0,0,0]
        else:
            scorecard[k,:] = getMotionScores(T1,T2)
    scorecard[t1,:] = [0,0,0,0,0,0,0,0,0]




    #print("Top scorers in Xpred,Ypred,Locscore:")
    k = min(3,nT)     #top3 unless there's 2 tubes lol
    toplists = []
    for s in range(0,k):
        D = scorecard[:,s]
        top = np.argpartition(D,-k)[-k:]
        toplists.append(top)
        print(top)

    toMerge = []
    for t in range(0,nT):
        if nT<3:
            if (t in toplists[0] or t in toplists[1]):
                toMerge.append(t)
        else:
            if (t in toplists[2]):
                toMerge.append(t)

    print("First toMerge",toMerge)
    for m in toMerge:
        if tubes[m][0][0] < (T1[-1][0]-min(A,len(T1)/4) ):
            toMerge.remove(m)
    toMerge = [m for m in toMerge if (m!=t1) and (m not in cantMerge)]
    WC = scorecard[toMerge]
    Scoreboard = np.zeros(len(toMerge))


    # GO THRU THE 5 POINTS! Add to scoreboard and see who wins.
    if not(toMerge):
        T1cantmerge=True
    else:
        # Best Location
        bestloc = np.argmax(WC[:,2])
        Scoreboard[bestloc] = Scoreboard[bestloc] + 1
        print(toMerge[bestloc], "wins for location")

        # Smallest velocity change
        bestvel = np.argmin((WC[:,6]-WC[:,5])**2 + (WC[:,8]-WC[:,7])**2)
        Scoreboard[bestvel] = Scoreboard[bestvel] + 1
        print(toMerge[bestvel], "wins for velocity")

        # Closest lineup
        bestcos = np.argmax((WC[:,3]+WC[:,4])/2 )
        Scoreboard[bestcos] = Scoreboard[bestcos] + 1
        print(toMerge[bestcos], "wins for lineup")

        # Best size matchup
        #Check around +/- 3 frames
        minax = sum([e[1][1][0] for e in T1[-A-3:-A+3]])/len([e[1][1][0] for e in T1[-A-3:-A+3]])
        majax = sum([e[1][1][1] for e in T1[-A-3:-A+3]])/len([e[1][1][1] for e in T1[-A-3:-A+3]])
        #minax = sum([e[1][1][0] for e in T1]) /len(T1)
        #majax = sum([e[1][1][1] for e in T1]) /len(T1)
        sizeboard = []
        for m in toMerge:
            #thislocminax = sum([e[1][1][0] for e in tubes[m][A-3:A+3]])/len([e[1][1][0] for e in tubes[m][A-3:A+3]])
            #thislocmajax = sum([e[1][1][1] for e in tubes[m][A-3:A+3]])/len([e[1][1][0] for e in tubes[m][A-3:A+3]])
            thislocminax = tubes[m][A][1][1][0]
            thislocmajax = tubes[m][A][1][1][1]
            #thisminax = sum([e[1][1][0] for e in tubes[m]])/len(tubes[m])
            #thismajax = sum([e[1][1][1] for e in tubes[m]])/len(tubes[m])
            sizeboard.append((thislocminax-minax)**2+(thislocmajax-majax)**2)
        bestsiz = np.argmin(sizeboard)
        Scoreboard[bestsiz] = Scoreboard[bestsiz] + 1
        print(toMerge[bestsiz], "wins for size")


        # Best spatial matchup 
        pixboard = []
        for m in toMerge:
            E1 = tubes[t1][-A]
            E2 = tubes[m][A]
            res = compareImgFromEllipse(E1,E2,useorb=True,useflann=0)
            pixboard.append(res)
        if pixboard[0] <= 1:
            bestpix = np.argmax(pixboard)
        else:
            bestpix = np.argmin(pixboard)
        Scoreboard[bestpix] = Scoreboard[bestpix] + 1
        print(toMerge[bestpix],"wins spatially")

    print("SCORE:")
    print("Possible merges", toMerge)
    print("Num of points",Scoreboard)
    print()
    mergeboard = [[toMerge[i],Scoreboard[i]] for i in range(0,len(toMerge)) ]


    # Make sure the remaining tubes are even mergeable
    endT1 = T1[-1][0]
    for m in mergeboard:
        #Gotta start around or after end of T1
        if tubes[m[0]][0][0] < (endT1-A):
            m[1] = -1

    #print(mergeboard)
    mergeboard.sort(key = lambda x: -x[1])
    print(mergeboard)
    print()


    #Now to determine final winner
    mergedFlag = False


    if not(mergeboard): #Check that it's not empty first
        print("Doesn't match to anything")
        Merges.append([t1,-1])
        continue


    leadscore = mergeboard[0][1]
    Lscore = scorecard[mergeboard[0][0],2]
    T2 = tubes[mergeboard[0][0]]
    #below is how many radii is the gap you merge across
    mergeradii = (((T1[-1][1][0][0] - T2[0][1][0][0])**2 + (T1[-1][1][0][1] - T2[0][1][0][1])**2)**0.5) / min(T1[-A][1][1][1],T2[A][1][1][1])
    Vs = scorecard[mergeboard[0][0],5:]

    if leadscore >= 5 and len(mergeboard)>1:
        #Gimme one last velocity check for extremes
        recalcVelocity(tubes, 10)
        negcount=0
        poscount=0
        Vx1 = sum([e[2][0] for e in T1[-A:-5]])/(A-5)
        Vy1 = sum([e[2][1] for e in T1[-A:-5]])/(A-5)
        Vys1 = [e[2][1] for e in T1]
        for v in Vys1:
            if v<0:
                negcount=negcount+1
            else:
                poscount=poscount+1
        Vx2 = sum([e[2][0] for e in tubes[mergeboard[0][0]][5:A]])/(A-5)
        Vy2 = sum([e[2][1] for e in tubes[mergeboard[0][0]][5:A]])/(A-5)
        Vys2 = [e[2][1] for e in tubes[mergeboard[0][0]] ]
        X1 = T1[-5][1][0][0]
        R1 = (T1[-5][1][1][1] + T1[-5][1][1][0])/2
        X2 = tubes[mergeboard[0][0]][5][1][0][0]
        Y1 = T1[-5][1][0][1]
        Y2 = tubes[mergeboard[0][0]][5][1][0][1]
        if (Vx1<-2 and X2>X1+R1) or (Vx1>2 and X2<X1-R1) or (negcount/len(Vys1)>0.9 and Y2>Y1+R1) or (poscount/len(Vys1)>0.9 and Y2<Y1-R1):
            print("Sorry, looks like that tube is just right behind you.")
            Merges.append([t1,-1])
        else: 
            print("This is a match.")
            print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[0][0])
            Merges.append([t1,mergeboard[0][0]])
            mergedFlag = True

    if leadscore<=0: 
        print("Doesn't match to anything")
        Merges.append([t1,-1])
        continue

    if leadscore<=1 and tubes[mergeboard[0][0]][0][0] - T1[-1][0] > 360:
        print("That's a laarge gap to merge across. I don't trust it, sorry.")
        Merges.append([t1,-1])
        continue

    if len(mergeboard) >1:
        if (leadscore == mergeboard[1][1]) and leadscore!= -1 :
            #We have a tie! 
            print("Tie goes to the closest in space.")
            print("THE WINNER (for tube ",t1,") IS: TUBE", toMerge[bestloc])
            Merges.append([t1,toMerge[bestloc]])
            T2 = tubes[toMerge[bestloc]]
            mergedFlag = True
    
    mergeradii = (((T1[-1][1][0][0] - T2[0][1][0][0])**2 + (T1[-1][1][0][1] - T2[0][1][0][1])**2)**0.5) / min(T1[-A][1][1][1],T2[A][1][1][1])
    if mergeradii>2.4 and T1[-A][1][1][1]>30:         
        print("Looking kinda far from the closest thing. Don't merge", t1)
        print(mergeradii,"far actually.")
        toDel = [m for m in Merges if m[0]==t1]
        Merges = [m for m in Merges if m not in toDel]
        Merges.append([t1,-1])
        mergedFlag = True
    elif T1[-A][1][1][1]<29:
        print("Actually, too small to merge")
        toDel = [m for m in Merges if m[0]==t1]
        Merges = [m for m in Merges if m not in toDel]
        Merges.append([t1,-1])
        mergedFlag = True
    
  
    if not mergedFlag:
        if (mergeboard[0][0] not in maybeMerge) or mergeboard[0][1]>=5 :
            #Gimme one last velocity check for extremes
            recalcVelocity(tubes, 10)
            negcount=0
            poscount=0
            Vx1 = sum([e[2][0] for e in T1[-A:-5]])/(A-5)
            Vy1 = sum([e[2][1] for e in T1[-A:-5]])/(A-5)
            Vys1 = [e[2][1] for e in T1]
            for v in Vys1:
                if v<0:
                    negcount=negcount+1
                else:
                    poscount=poscount+1
            Vx2 = sum([e[2][0] for e in tubes[mergeboard[0][0]][5:A]])/(A-5)
            Vy2 = sum([e[2][1] for e in tubes[mergeboard[0][0]][5:A]])/(A-5)
            Vys2 = [e[2][1] for e in tubes[mergeboard[0][0]] ]
            X1 = T1[-5][1][0][0]
            R1 = (T1[-5][1][1][1] + T1[-5][1][1][0])/2
            X2 = tubes[mergeboard[0][0]][5][1][0][0]
            Y1 = T1[-5][1][0][1]
            Y2 = tubes[mergeboard[0][0]][5][1][0][1]

            if (mergeboard[0][1]<5) and ((Vx1/Vx2<=0) and (abs(Vx1)+abs(Vx2)>=5)) or ((Vy1/Vy2<=0) and (abs(Vy1)+abs(Vy2)>=7)):
                print("Sorry, looks like that tube is going the opposite direction.")
                Merges.append([t1,-1])
            elif (Vx1<-2 and X2>X1+R1) or (Vx1>2 and X2<X1-R1) or (negcount/len(Vys1)>0.9 and Y2>Y1+R1) or (poscount/len(Vys1)>0.9 and Y2<Y1-R1):
                print("Sorry, looks like that tube is just right behind you.")
                Merges.append([t1,-1])
            else:
                print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[0][0])
                Merges.append([t1,mergeboard[0][0]])
                mergedFlag = True
        else:
            #One last check if the second thing is okay.
            MB = [m for m in mergeboard if m[1]!=-1]
            mb = [m[0] in maybeMerge for m in MB]

            if False in mb: #something isn't in maybeMerge
                T2 = tubes[mergeboard[mb.index(False)][0]]
                mergeradii = (((T1[-A][1][0][0] - T2[0][1][0][0])**2 + (T1[-A][1][0][1] - T2[0][1][0][1])**2)**0.5) / T1[-A][1][1][1]
                print("Merge radius:", mergeradii)
                if mergeradii < 1 and (mergeboard[mb.index(False)][1] >= 2):
                    print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[mb.index(False)][0])
                    Merges.append([t1,mergeboard[mb.index(False)][0]])
                    mergedFlag = True
                else: #not a great score
                    print("Doesn't match to anything")
                    Merges.append([t1,-1])     

            else: #everything is in maybeMerge
                T2 = tubes[mergeboard[0][0]]
                mergeradii = (((T1[-A][1][0][0] - T2[0][1][0][0])**2 + (T1[-A][1][0][1] - T2[0][1][0][1])**2)**0.5) / T1[-A][1][1][1]
                print("Merge radius:", mergeradii)
                print(mergeboard[0][0])
                print(getMotionScores(T1,T2))
                Vs = scorecard[mergeboard[0][0],5:]
                print("Vs:", Vs)
                if mergeradii<1 and ((Vs[0]/Vs[1]>0) or (abs(Vs[0])+abs(Vs[1])<3)) and ((Vs[2]/Vs[3]>0) or (abs(Vs[2])+abs(Vs[3])<3)) : #score and vel are ok
                    print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[0][0])
                    Merges.append([t1,mergeboard[0][0]])
                    mergedFlag = True 
                else:
                    print("Doesn't match to anything")
                    Merges.append([t1,-1])


    if mergedFlag:
        check = [m for m in Merges if m[0]==t1][0]
        T2 = tubes[check[1]]
        if (T1[-1][0]-T2[0][0]>min(len(T1)/3,len(T2)/3)):
            print("Those kinda overlap.")
            toDel = [m for m in Merges if m[0]==t1]
            Merges = [m for m in Merges if m not in toDel]
            Merges.append([t1,-1])
            mergedFlag = True            
            

print(Merges)

#--------------------------------------------------------------------------------
#Actually do the merge


print(Merges)
A=10
sectubes = [m[1] for m in Merges]
two2one = [t for t in range(0,nT) if sectubes.count(t)>1]
toAdd= []
for t in two2one:
    tm = [m for m in Merges if m[1]==t]
    if len(tm)==2:
        A1 = [e[0] for e in tubes[tm[0][0]]]
        A2 = [e[0] for e in tubes[tm[1][0]]]
        overlap = [val for val in A1 if val in A2]
        if len(overlap)>15:
            #Merges = [M for M in Merges if M not in tm] 
            #NAW WE NEED TO CHOOSE THE CLOSEST!
            checke = tubes[tm[1][1]][A]
            B1 = [(e[1][0][0]-checke[1][0][0])**2+(e[1][0][1]-checke[1][0][1])**2 for e in tubes[tm[0][0]][-A-3:-A+3]]
            B2 = [(e[1][0][0]-checke[1][0][0])**2+(e[1][0][1]-checke[1][0][1])**2 for e in tubes[tm[1][0]][-A-3:-A+3]]
            if min(B1)<min(B2):
                toAdd.append([tm[0][0],tm[1][1]])
            else:
                toAdd.append([tm[1][0],tm[1][1]])
            ##NEED TO CHOOSE smaller ONE
            #B1 = [e[1][1][1] for e in tubes[tm[0][0]]]
            #B2 = [e[1][1][1] for e in tubes[tm[1][0]]]
            #if max(B2)<max(B1):
            #    toAdd.append([tm[1][0],tm[1][1]])
            #else:
            #    toAdd.append([tm[0][0],tm[1][1]])
        else:
            #chain them
            if A1[0]<A2[0]:
                toAdd.append([tm[0][0],tm[1][0],tm[1][1]])
            else:
                toAdd.append([tm[1][0],tm[0][0],tm[1][1]])
        Merges = [M for M in Merges if M not in tm]   
    else:
        Merges = [M for M in Merges if M not in tm] 

if toAdd:
    for ta in toAdd:
        Merges.append(ta)

print(Merges)
mergechains = []
#Turn Merges into chain
for m in Merges:
    if m[1]==-1:
        continue
    else:
        M1s = [mc[-1] for mc in mergechains]
        if m[0] in M1s:
            mergechains[M1s.index(m[0])].append(m[1])
        else:
            mergechains.append(m)
print(mergechains)


toDel = []
for m in mergechains:
    supertube = []
    checkpts = [tubes[t][A][0]  for t in m] + [tubes[t][-A][0]  for t in m]
    checkpts.sort()
    for t in m:
        toDel.append(t)
        supertube.extend(tubes[t])
    supertube.sort(key = lambda x: x[0])
    
    #do supertube colors
    RGB = tubes[m[0]][0][3]
    for e in supertube:
        e[3] = RGB
    
    thefinal = supertube[-1][0]
    for thisf in range(int(supertube[0][0]), int(supertube[-1][0])):
        ishere = [e for e in supertube if e[0]==thisf]
        if ishere:
            continue
        else:
            low = thisf
            high = thisf
            while low not in checkpts and low>supertube[0][0]:
                low=low-1
            #print("The last", supertube[-1][0])
            while (high not in checkpts) and high<thefinal :
                high=high+1
                
            fgap = high-low
            laste = [e for e in supertube if e[0]==low][0]
            nexte = [e for e in supertube if e[0]==high][0]
            
            X = laste[1][0][0] + (nexte[1][0][0]-laste[1][0][0])*((thisf-low)/fgap)
            Y = laste[1][0][1] + (nexte[1][0][1]-laste[1][0][1])*((thisf-low)/fgap)
            smallax = laste[1][1][0] + (nexte[1][1][0]-laste[1][1][0])*((thisf-low)/fgap)
            bigax = laste[1][1][1] + (nexte[1][1][1]-laste[1][1][1])*((thisf-low)/fgap)
            phi = laste[1][2]
            Vx = laste[2][0] + (nexte[2][0]-laste[2][0])*((thisf-low)/fgap)
            Vy = laste[2][1] + (nexte[2][1]-laste[2][1])*((thisf-low)/fgap)
        
            thisslice = [thisf, ((X,Y),(smallax,bigax),phi), [Vx,Vy], RGB]
            supertube.append(thisslice)
            
            
    #Add supertube. subtract old tubes
    tubes.append(supertube)
    
tubes = [T for T in tubes if tubes.index(T) not in toDel]

#quick color wash for double cases
for t in tubes:
    RGB = t[0][3]
    for e in t:
        e[3] = RGB

#----------------------------------------------------------------------
 
END = time.time()
print()
print("Time:", END-START)
print()
putTubesOnVid(tubes, vidbase+'_outFINAL.mp4', True)



<class 'list'>
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184
6357
Tube14
0
0
[6, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 31, 32, 33, 34, 35, 36, 40, 41, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66, 71, 72, 73, 74, 75, 76, 81, 82, 83, 84, 85, 86, 91, 92, 93, 94, 95, 96, 101, 102, 103, 104, 105, 106, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126, 131, 132, 133, 134, 135, 136, 140, 141, 142, 143, 144, 145, 146, 151, 152, 153, 154, 155, 156, 161, 162, 163, 164, 165, 166, 171, 172, 173, 174, 175, 176, 181, 182, 183, 184, 185, 186, 191, 192, 193, 194, 195, 196, 201, 202, 203, 204, 205, 206, 211, 212, 213, 214, 215, 216, 221, 222, 223, 224, 225, 226, 231, 232, 233, 234, 235, 236, 241, 242, 243, 244, 245, 246]
[40, 41, 42, 43, 44, 45, 46]

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27

Tube  28 [12, 10] . Length =  200 Score1:  0.05 Score2: 118.71517443656921
Tube  29 [24, 21] . Length =  91 Score1:  0.23076923076923078 Score2: 0.9650319417317803
Tube 29 shouldn't engulf or be engulfed
Tube  30 [5, 1] . Length =  34 Score1:  0.029411764705882353 Score2: 3.9332758585611742
Tube 30 shouldn't engulf or be engulfed
Tube  31 [4, 5] . Length =  44 Score1:  0.09090909090909091 Score2: 9.076126098632812
Tube 31 shouldn't engulf or be engulfed
[4, 7, 8, 9, 10, 13, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 29, 30, 31]
We got a hit!
First tube is  2 and swallowing tube is  28
Average position delta: 0.9159591781985921
Here it's:  3.474703954617395
Average size delta: 0.8286505451844508
Here it's  8.965964428094955
At least part of the track on 2 was too steady to trust.


C:\Users\kjarrett\Anaconda3\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\kjarrett\Anaconda3\envs\py36\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Done.
End of S6
[5, 11, 12, 15]

Has subtubes: [5, 11, 12, 15]
Has glides: [4, 5, 6, 11, 16, 28]
Has steadys: [3, 7]
0.0 on tube 3  subtube 0
0.015151515151515152 on tube 3  subtube 1
0.0 on tube 3  subtube 2
0.0 on tube 5  subtube 0
0.0 on tube 5  subtube 1
0.0 on tube 5  subtube -1
0.0 on tube 7  subtube 0
0.0 on tube 7  subtube 1
0.05 on tube 7  subtube 2
0.0 on tube 11  subtube 0
0.0 on tube 11  subtube 1
0.0 on tube 11  subtube 2
0.0 on tube 11  subtube -1
0.07407407407407407 on tube 12  subtube 0
0.03389830508474576 on tube 12  subtube 1
0.10650887573964497 on tube 15  subtube 0
0.0 on tube 15  subtube 1
Whole tubes to delete: []
[5, 11, 12, 15]
[[25, 358], [10, 11, 174], [27, 59], [169, 28]]
[[0, 0], [0, 0, 0], [0, 0], [0, 0]]

Going backward on tube 12 ...
X score is: 0.9727365065909902
Y score is: 0.9029235187861171
X cosine score 0.9892551652421838
Y cosine score 0.967647179796334
FINAL: 0.9755938395551245

Going forward on tube  15 ...
X score is: 0.4651001184775854
Y score 

FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [15]:
# Tracking Test

tubes = readTubelist("ucr3_s5Tubelist.csv")
V = readvid("ucr3.mp4")

L = deepcopy(tubes)
tracks = []

START = time.time()

for t in L:
    
    if len(t)<20:
        continue
    
    print("Tube ",L.index(t))
    
    startE = t[15]
    startf = startE[0]
    
    tracker = cv2.TrackerCSRT_create()
    starttrack = []
    endtrack = []
    

    bbox = getBoxFromEllipse(startE)
    ok = tracker.init(V[int(startf)], bbox)
    starttrack.append((startf, bbox))
    
    k=1
    while ok and startf+k<F-1:

        ok, bbox = tracker.update(V[int(startf+k)])
        if ok:
            starttrack.append((startf+k, bbox))
        k=k+1
                   

            
    tracks.append(starttrack)

END = time.time()

print()
print("Time:", END-START)
print()
#putTracksOnVid2(tracks,vidbase+'.mp4',vidbase+'_KCFTEST.mp4')

<class 'list'>
Tube  0
Tube  1
Tube  2
Tube  3
Tube  4
Tube  5
Tube  6
Tube  7
Tube  8
Tube  9
Tube  10
Tube  11
Tube  12
Tube  13
Tube  14
Tube  15
Tube  16
Tube  17
Tube  18
Tube  19
Tube  20
Tube  21
Tube  22
Tube  23
Tube  24
Tube  25
Tube  26
Tube  27
Tube  28
Tube  29
Tube  31
Tube  32
Tube  33
Tube  34
Tube  35
Tube  36
Tube  38
Tube  39
Tube  40
Tube  41

Time: 490.53022933006287

